In [215]:
import numpy as np
import pandas as pd
from scipy.stats import percentileofscore
import uuid
import json
import os
import urllib
import plotly.plotly as py
import plotly.graph_objs as go

In [216]:
pd.options.display.max_columns = None

# Loading, Cleaning and Organizing Relevant Datasets

We will load **HIES Survey Data** (PSLM Household Integrated Economic Survey) and **Tax Data**

## HIES Survey Data

We load microdata from years 2004 - 2016, with a gap between 2009 - 2010. The names of the relevant tables that we create after running the code below are listed below. See the bottom of this section (after all the intermediate calculations and tables) for code to download these tables.

Here we have the intermediate calculations and tables used to pull together this data. We show the first five rows of each table created so we can get an idea of what the data looks actually like.

### 2004 - 2005

In [217]:
weights_2004_05 = pd.read_stata('./heisSurvey/2004-05/weight file.dta')
weights_2004_05.drop(["v3"], inplace=True, axis=1)
weights_2004_05[0:5]

,sno,psu,weight,quarter,region,psu_hies
0,1,10011002,145.544006,1,1,11711101
1,2,10011004,178.582001,2,1,11711201
2,3,10011005,142.528000,3,1,11711301
3,4,10012004,880.974976,1,1,11712101
4,5,10012007,964.565002,2,1,11712201


In [218]:
unweighted_hh_balance_sheet_2004_05 = pd.read_stata('./heisSurvey/2004-05/sec_n0.dta')
unweighted_hh_balance_sheet_2004_05["hhcode"] = unweighted_hh_balance_sheet_2004_05["hhcode"].astype(int)
unweighted_hh_balance_sheet_2004_05["hhcode_hies"] = unweighted_hh_balance_sheet_2004_05["hhcode_hies"].astype(int)
unweighted_hh_balance_sheet_2004_05 = unweighted_hh_balance_sheet_2004_05[["hhcode", "hhcode_hies", "msno", "n1_12", "n2_6", "psu", "region", "psu_hies", "hhno", "province"]]
unweighted_hh_balance_sheet_2004_05.rename(index=str, inplace=True, columns={
        "n1_12": "income",
        "n2_6": "expenditure"
})
unweighted_hh_balance_sheet_2004_05[0:5]

,hhcode,hhcode_hies,msno,income,expenditure,psu,region,psu_hies,hhno,province
0,1001100201,1171110101,0,160800.0,111870,10011002,urban,11711101,1,punjab
1,1001100202,1171110102,0,67200.0,55022,10011002,urban,11711101,2,punjab
2,1001100203,1171110103,0,115800.0,101880,10011002,urban,11711101,3,punjab
3,1001100204,1171110104,0,202800.0,117448,10011002,urban,11711101,4,punjab
4,1001100205,1171110105,0,617400.0,272012,10011002,urban,11711101,5,punjab


Note this is not capital income + individual income of members of household. See the survey from this year, they do not include individuals who don't spend most of their income on household expenses. The exact phrase is : "If he/she did not spend most of his income on household expenses, then do not include his/her income in the Family‟s overall income"

In [219]:
# HOUSEHOLD BALANCE SHEET
hh_balance_sheet_2004_05 = unweighted_hh_balance_sheet_2004_05.join(weights_2004_05[["psu", "weight"]].set_index('psu'), on='psu')[["hhcode", "income", "expenditure", "psu", "region", "psu_hies", "hhno", "hhcode_hies", "province", "weight"]]
hh_balance_sheet_2004_05.rename(index=str, inplace=True, columns={
        "weight": "weights"
})
hh_balance_sheet_2004_05[0:5]

,hhcode,income,expenditure,psu,region,psu_hies,hhno,hhcode_hies,province,weights
0,1001100201,160800.0,111870,10011002,urban,11711101,1,1171110101,punjab,145.544006
1,1001100202,67200.0,55022,10011002,urban,11711101,2,1171110102,punjab,145.544006
2,1001100203,115800.0,101880,10011002,urban,11711101,3,1171110103,punjab,145.544006
3,1001100204,202800.0,117448,10011002,urban,11711101,4,1171110104,punjab,145.544006
4,1001100205,617400.0,272012,10011002,urban,11711101,5,1171110105,punjab,145.544006


In [220]:
unweighted_individual_balance_sheet_2004_05 = pd.read_stata('./heisSurvey/2004-05/sec_e0.dta')
unweighted_individual_balance_sheet_2004_05["hhcode_hies"] = unweighted_individual_balance_sheet_2004_05["hhcode_hies"].astype(int)
unweighted_individual_balance_sheet_2004_05["hhcode"] = unweighted_individual_balance_sheet_2004_05["hhcode"].astype(int)
unweighted_individual_balance_sheet_2004_05.drop(["sec", "seq01", "seq02", "seq03", "seq04", "seq05", "seq06", "seq07", "seq08", "seq09", "seq11", "seq12", "seq15"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2004_05.rename(index=str, inplace=True, columns={
        "seq10": "industry_sector",
        "seq13": "monthly_income", 
        "seq14": "months_worked", 
        "seq16": "other_annual_earnings"
})
unweighted_individual_balance_sheet_2004_05["hhcode"] = unweighted_individual_balance_sheet_2004_05["hhcode"].astype(int)
unweighted_individual_balance_sheet_2004_05["monthly_income"] = unweighted_individual_balance_sheet_2004_05["monthly_income"].fillna(0)
unweighted_individual_balance_sheet_2004_05["months_worked"] = unweighted_individual_balance_sheet_2004_05["months_worked"].fillna(0)
unweighted_individual_balance_sheet_2004_05["other_annual_earnings"] = unweighted_individual_balance_sheet_2004_05["other_annual_earnings"].fillna(0)
unweighted_individual_balance_sheet_2004_05["income"] = (unweighted_individual_balance_sheet_2004_05["monthly_income"] * unweighted_individual_balance_sheet_2004_05["months_worked"]) + unweighted_individual_balance_sheet_2004_05["other_annual_earnings"]
unweighted_individual_balance_sheet_2004_05.drop(["monthly_income", "months_worked", "other_annual_earnings"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2004_05 = unweighted_individual_balance_sheet_2004_05[pd.notnull(unweighted_individual_balance_sheet_2004_05["income"])]
unweighted_individual_balance_sheet_2004_05 = unweighted_individual_balance_sheet_2004_05[unweighted_individual_balance_sheet_2004_05["income"] != 0]
unweighted_individual_balance_sheet_2004_05[0:5]

,hhcode,msno,industry_sector,psu,quarter,region,psu_hies,hhno,hhcode_hies,province,income
0,1001100201,1,4 electricity,10011002,1,urban,11711101,1,1171110101,punjab,94800.0
2,1001100201,3,9 social & personal service,10011002,1,urban,11711101,1,1171110101,punjab,66000.0
4,1001100202,1,10 other,10011002,1,urban,11711101,2,1171110102,punjab,67200.0
6,1001100203,1,10 other,10011002,1,urban,11711101,3,1171110103,punjab,55800.0
12,1001100204,1,10 other,10011002,1,urban,11711101,4,1171110104,punjab,80400.0


In [221]:
individual_characteristics_2004_05 = pd.read_stata('./heisSurvey/2004-05/sec_b0.dta')
individual_characteristics_2004_05["hhcode"] = individual_characteristics_2004_05["hhcode"].astype(int)
individual_characteristics_2004_05.drop(["sec", "sbq02", "sbq05", "quarter", "province", "region", "psu", "psu_hies", "hhno", "hhcode_hies"], inplace=True, axis=1)
individual_characteristics_2004_05.rename(index=str, inplace=True, columns={
        "sbq01": "sex", 
        "sbq03": "relation_to_head", 
        "sbq04": "age"
})
individual_characteristics_2004_05["birth_year"] = 2019 - individual_characteristics_2004_05["age"]
individual_characteristics_2004_05[0:5]

,hhcode,msno,sex,relation_to_head,age,weight,birth_year
0,1001100201,1,male,1:self,50,145.544006,1969
1,1001100201,2,female,2: husband/wife,46,145.544006,1973
2,1001100201,3,female,3:son/daughter,24,145.544006,1995
3,1001100201,4,male,3:son/daughter,17,145.544006,2002
4,1001100202,1,male,1:self,40,145.544006,1979


In [222]:
# INDIVIDUAL BALANCE SHEET
individual_balance_sheet_2004_05 = unweighted_individual_balance_sheet_2004_05.join(individual_characteristics_2004_05.set_index(["hhcode", "msno"]), on=["hhcode", "msno"])
individual_balance_sheet_2004_05.rename(index=str, inplace=True, columns={
        "weight": "weights"
})
print("num_rows", individual_balance_sheet_2004_05.shape[0])
individual_balance_sheet_2004_05[0:5]

num_rows 21170


,hhcode,msno,industry_sector,psu,quarter,region,psu_hies,hhno,hhcode_hies,province,income,sex,relation_to_head,age,weights,birth_year
0,1001100201,1,4 electricity,10011002,1,urban,11711101,1,1171110101,punjab,94800.0,male,1:self,50,145.544006,1969
2,1001100201,3,9 social & personal service,10011002,1,urban,11711101,1,1171110101,punjab,66000.0,female,3:son/daughter,24,145.544006,1995
4,1001100202,1,10 other,10011002,1,urban,11711101,2,1171110102,punjab,67200.0,male,1:self,40,145.544006,1979
6,1001100203,1,10 other,10011002,1,urban,11711101,3,1171110103,punjab,55800.0,male,1:self,33,145.544006,1986
12,1001100204,1,10 other,10011002,1,urban,11711101,4,1171110104,punjab,80400.0,male,1:self,52,145.544006,1967


In [223]:
# Note the psu = 4160003 does not have the weight filled in correctly, we do this manually in the next row
set(individual_balance_sheet_2004_05[pd.isnull(individual_balance_sheet_2004_05["weights"])]["psu"])

{41610003}

In [224]:
weights_2004_05[weights_2004_05["psu"] == 41610003]["weight"]

993    231.035995
Name: weight, dtype: float32

In [225]:
for i in range(0, individual_balance_sheet_2004_05.shape[0]):
    if pd.isnull(individual_balance_sheet_2004_05["weights"][i]) and individual_balance_sheet_2004_05["psu"][i] == 41610003:
        individual_balance_sheet_2004_05["weights"][i] = 231.035995

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [226]:
print(set(individual_balance_sheet_2004_05[pd.isnull(individual_balance_sheet_2004_05["weights"])]["psu"]))
individual_balance_sheet_2004_05[0:5]

set()


,hhcode,msno,industry_sector,psu,quarter,region,psu_hies,hhno,hhcode_hies,province,income,sex,relation_to_head,age,weights,birth_year
0,1001100201,1,4 electricity,10011002,1,urban,11711101,1,1171110101,punjab,94800.0,male,1:self,50,145.544006,1969
2,1001100201,3,9 social & personal service,10011002,1,urban,11711101,1,1171110101,punjab,66000.0,female,3:son/daughter,24,145.544006,1995
4,1001100202,1,10 other,10011002,1,urban,11711101,2,1171110102,punjab,67200.0,male,1:self,40,145.544006,1979
6,1001100203,1,10 other,10011002,1,urban,11711101,3,1171110103,punjab,55800.0,male,1:self,33,145.544006,1986
12,1001100204,1,10 other,10011002,1,urban,11711101,4,1171110104,punjab,80400.0,male,1:self,52,145.544006,1967


### 2005 - 2006

In [227]:
weights_2005_06 = pd.read_stata('./heisSurvey/2005-06/p list.dta')
weights_2005_06["hhcode"] = weights_2005_06["hhcode"].astype(int)
weights_2005_06 = weights_2005_06[["hhcode", "idc", "s1aq02", "s1aq03", "s1aq05c", "weight", "region", "province"]]
weights_2005_06.rename(index=str, inplace=True, columns={
        "s1aq02": "relation_to_head",
        "s1aq03": "sex",
        "s1aq05c": "birth_year"
})
weights_2005_06[0:5]

,hhcode,idc,relation_to_head,sex,birth_year,weight,region,province
0,1011010101,1,head,male,1941.0,1031.644,urban,punjab
1,1011010101,51,spouse,female,1947.0,1031.644,urban,punjab
2,1011010102,1,head,male,1969.0,1031.644,urban,punjab
3,1011010102,51,spouse,female,1975.0,1031.644,urban,punjab
4,1011010102,52,son/daughter,male,1997.0,1031.644,urban,punjab


In [228]:
unweighted_hh_balance_sheet_2005_06 = pd.read_stata('./heisSurvey/2005-06/sec 12c.dta')
unweighted_hh_balance_sheet_2005_06["hhcode"] = unweighted_hh_balance_sheet_2005_06["hhcode"].astype(int)
unweighted_hh_balance_sheet_2005_06.rename(index=str, inplace=True, columns={
        "s12cq01": "income", 
        "s12cq02": "expenditure", 
        "s12cq03": "ratio", 
        "s12cq04": "does_ratio_make_sense"
})
print("num_rows", unweighted_hh_balance_sheet_2005_06.shape[0])
unweighted_hh_balance_sheet_2005_06[0:5]

num_rows 15450


,hhcode,income,expenditure,ratio,does_ratio_make_sense
0,1011010101,38400.0,59524.0,0.64,no
1,1011010102,144000.0,147498.0,0.98,yes
2,1011010103,58000.0,57900.0,1.00,yes
3,1011010104,81000.0,122634.0,0.67,no
4,1011010105,120000.0,103502.0,1.16,yes


In [229]:
# If the ratio above did not make sense, a more through accounting involving transfers, 
# buildings, financial assets, agricultural assets etc were taken into account.
# See the introduction sheet in ./heisSurvey/2005-06/ for more information 
capital_2005_06 = pd.read_stata('./heisSurvey/2005-06/sec 12e.dta')
capital_2005_06["hhcode"] = capital_2005_06["hhcode"].astype(int)
capital_2005_06.rename(index=str, inplace=True, columns={
        "s12eq01": "income", 
        "s12eq02": "expenditure", 
        "s12eq03": "ratio", 
        "s12eq04": "does_ratio_make_sense"
})
print("num_rows", capital_2005_06.shape[0])
capital_2005_06[0:5]

num_rows 3625


,hhcode,income,expenditure,ratio,does_ratio_make_sense
0,1011010101,54400,60374.0,0.90,yes
1,1011010104,82500,124384.0,0.66,no
2,1011010107,294000,289720.0,1.01,yes
3,1011010112,192000,184882.0,1.03,yes
4,1011010202,116000,120994.0,0.96,yes


In [230]:
# replace incomes in "unweighted_hh_balance_sheet_2005_06" with those in "capital_2005_06", as they are more accurate
for i in range(0, capital_2005_06.shape[0]):
    hhcode = capital_2005_06["hhcode"][i]
    new_income = capital_2005_06["income"][i]
    new_expenditure = capital_2005_06["expenditure"][i]
    new_ratio = capital_2005_06["ratio"][i]
    new_message = 'yes' if new_ratio >= 0.85 else 'no'
    
    row_index = unweighted_hh_balance_sheet_2005_06.index[unweighted_hh_balance_sheet_2005_06['hhcode'] == hhcode].tolist()[0]
    unweighted_hh_balance_sheet_2005_06.at[row_index, "income"] = new_income
    unweighted_hh_balance_sheet_2005_06.at[row_index, "expenditure"] = new_expenditure
    unweighted_hh_balance_sheet_2005_06.at[row_index, "ratio"] = new_ratio
    unweighted_hh_balance_sheet_2005_06.at[row_index, "does_ratio_make_sense"] = new_message

In [231]:
# HOUSEHOLD BALANCE SHEET
weights_2005_06_subset = weights_2005_06[["hhcode", "weight", "region", "province"]].drop_duplicates().set_index('hhcode')
hh_balance_sheet_2005_06 = unweighted_hh_balance_sheet_2005_06.join(weights_2005_06_subset, on='hhcode')
hh_balance_sheet_2005_06.rename(index=str, inplace=True, columns={
        "weight": "weights" 
})
hh_balance_sheet_2005_06[0:5]

,hhcode,income,expenditure,ratio,does_ratio_make_sense,weights,region,province
0,1011010101,54400.0,60374.0,0.90,yes,1031.644,urban,punjab
1,1011010102,144000.0,147498.0,0.98,yes,1031.644,urban,punjab
2,1011010103,58000.0,57900.0,1.00,yes,1031.644,urban,punjab
3,1011010104,82500.0,124384.0,0.66,no,1031.644,urban,punjab
4,1011010105,120000.0,103502.0,1.16,yes,1031.644,urban,punjab


In [232]:
unweighted_individual_balance_sheet_2005_06 = pd.read_stata('./heisSurvey/2005-06/sec 12a.dta')
unweighted_individual_balance_sheet_2005_06["hhcode"] = unweighted_individual_balance_sheet_2005_06["hhcode"].astype(int)
unweighted_individual_balance_sheet_2005_06.drop(["s12aq01", "s12aq02", "s12aq03", "s12aq04", "s12aq05", "s12aq06", "s12aq07"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2005_06.rename(index=str, inplace=True, columns={
        "s12aq08": "income"
})
unweighted_individual_balance_sheet_2005_06[0:5]

,hhcode,province,region,idc,income
0,1011010101,punjab,urban,1,38400.0
1,1011010101,punjab,urban,99,38400.0
2,1011010102,punjab,urban,1,144000.0
3,1011010102,punjab,urban,99,144000.0
4,1011010103,punjab,urban,1,48000.0


In [233]:
# INDIVIDUAL BALANCE SHEET
work_characteristics_2005_06 = pd.read_stata('./heisSurvey/2005-06/sec 1b.dta')
work_characteristics_2005_06 = work_characteristics_2005_06[["s1bq05", "idc", "hhcode"]]
work_characteristics_2005_06.rename(index=str, inplace=True, columns={
        "s1bq05": "industry_sector"
})

temp = unweighted_individual_balance_sheet_2005_06.merge(work_characteristics_2005_06, how="inner", on=["hhcode", "idc"])
individual_balance_sheet_2005_06 = temp.merge(weights_2005_06, how="left", on=["hhcode", "idc"])
individual_balance_sheet_2005_06.drop(["province_y", "region_y"], inplace=True, axis=1),
individual_balance_sheet_2005_06.rename(index=str, inplace=True, columns={
        "province_x": "province",
        "region_x": "region",
        "weight": "weights"
})
individual_balance_sheet_2005_06[0:5]

,hhcode,province,region,idc,income,industry_sector,relation_to_head,sex,birth_year,weights
0,1011010101,punjab,urban,1,38400.0,NaN,head,male,1941.0,1031.644
1,1011010102,punjab,urban,1,144000.0,61.0,head,male,1969.0,1031.644
2,1011010103,punjab,urban,1,48000.0,62.0,head,male,1934.0,1031.644
3,1011010103,punjab,urban,2,10000.0,62.0,son/daughter,male,1984.0,1031.644
4,1011010104,punjab,urban,1,81000.0,41.0,head,male,1965.0,1031.644


In [234]:
# Note these individuals didn't have weights filled in (data entry error)
individual_balance_sheet_2005_06[pd.isnull(individual_balance_sheet_2005_06["weights"])]

,hhcode,province,region,idc,income,industry_sector,relation_to_head,sex,birth_year,weights
9114,1282010111,punjab,rural,1,44000.0,11.0,NaN,NaN,NaN,NaN
18582,3041020103,nwfp,urban,54,14400.0,93.0,NaN,NaN,NaN,NaN
18609,3041020201,nwfp,urban,2,108000.0,NaN,NaN,NaN,NaN,NaN
19658,3072030105,nwfp,rural,3,55500.0,62.0,NaN,NaN,NaN,NaN
20600,3122010102,nwfp,rural,52,12000.0,11.0,NaN,NaN,NaN,NaN
20614,3122010116,nwfp,rural,1,16000.0,11.0,NaN,NaN,NaN,NaN


In [235]:
for i in range(0, individual_balance_sheet_2005_06.shape[0]):
    if pd.isnull(individual_balance_sheet_2005_06["weights"][i]):
        hhcode = individual_balance_sheet_2005_06["hhcode"][i]
        weight = list(set(weights_2005_06[weights_2005_06["hhcode"] == hhcode]["weight"]))[0]
        individual_balance_sheet_2005_06["weights"][i] = weight

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [236]:
print(set(individual_balance_sheet_2005_06[pd.isnull(individual_balance_sheet_2005_06["weights"])]["hhcode"]))
individual_balance_sheet_2005_06[0:5]

set()


,hhcode,province,region,idc,income,industry_sector,relation_to_head,sex,birth_year,weights
0,1011010101,punjab,urban,1,38400.0,NaN,head,male,1941.0,1031.644
1,1011010102,punjab,urban,1,144000.0,61.0,head,male,1969.0,1031.644
2,1011010103,punjab,urban,1,48000.0,62.0,head,male,1934.0,1031.644
3,1011010103,punjab,urban,2,10000.0,62.0,son/daughter,male,1984.0,1031.644
4,1011010104,punjab,urban,1,81000.0,41.0,head,male,1965.0,1031.644


In [237]:
# Note these individuals didn't have income filled in (data entry error)
individual_balance_sheet_2005_06[pd.isnull(individual_balance_sheet_2005_06["income"])][0:5]

,hhcode,province,region,idc,income,industry_sector,relation_to_head,sex,birth_year,weights
90,1011020309,punjab,urban,1,NaN,91.0,head,male,1956.0,1799.566
172,1011040204,punjab,urban,51,NaN,62.0,head,female,1945.0,834.001
174,1011040205,punjab,urban,5,NaN,71.0,son/daughter,male,1980.0,834.001
192,1011040310,punjab,urban,2,NaN,51.0,brother/sister,male,1982.0,1319.725
193,1011040310,punjab,urban,3,NaN,51.0,brother/sister,male,1987.0,1319.725


In [238]:
z = pd.read_stata('./heisSurvey/2005-06/sec 1b.dta')
z.rename(index=str, inplace=True, columns={
        "s1bq05": "industry_sector",
        "s1bq08": "monthly_income", 
        "s1bq09": "months_worked", 
        "s1bq10": "annual_earnings",
        "s1bq15": "other_annual_earnings",
        "s1bq17": "other_other_annual_earnings",
        "s1bq19": "selling_wages_annual_earnings",
        "s1bq21": "pension_annual_earnings"
})
z["hhcode"] = z["hhcode"].astype(int)

# replace NaNs with 0s
z.monthly_income.fillna(0, inplace=True)
z.months_worked.fillna(0, inplace=True)
z.annual_earnings.fillna(0, inplace=True)
z.other_annual_earnings.fillna(0, inplace=True)
z.other_annual_earnings.fillna(0, inplace=True)
z.other_other_annual_earnings.fillna(0, inplace=True)
z.selling_wages_annual_earnings.fillna(0, inplace=True)
z.pension_annual_earnings.fillna(0, inplace=True)

for i in range(0, individual_balance_sheet_2005_06.shape[0]):
    if pd.isnull(individual_balance_sheet_2005_06["income"][i]):
        hhcode = individual_balance_sheet_2005_06["hhcode"][i]
        idc = individual_balance_sheet_2005_06["idc"][i]
        # find right row
        x = z[np.logical_and(z["idc"] == idc, z["hhcode"] == hhcode)]
        # calculate income
        income = (x["monthly_income"] * x["months_worked"]) + \
            x["annual_earnings"] + \
            x["other_annual_earnings"] + \
            x["other_other_annual_earnings"] + \
            x["selling_wages_annual_earnings"] + \
            x["pension_annual_earnings"]
        individual_balance_sheet_2005_06["income"][i] = income

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [239]:
# No individuals with no income reported
individual_balance_sheet_2005_06[pd.isnull(individual_balance_sheet_2005_06["income"])][0:5]

,hhcode,province,region,idc,income,industry_sector,relation_to_head,sex,birth_year,weights


### 2006 - 2007

In [240]:
weights_2006_07 = pd.read_stata('./heisSurvey/2006-07/hhweights.dta')
weights_2006_07["hhcode"] = weights_2006_07["hhcode"].astype(int)
weights_2006_07[0:5]

,hhcode,weights
0,1001100101,104.007
1,1001100102,104.007
2,1001100103,104.007
3,1001100104,104.007
4,1001100105,104.007


Here we need to create individual and household balance sheets as that currently does not exist

In [241]:
unweighted_individual_balance_sheet_2006_07 = pd.read_stata('./heisSurvey/2006-07/section e.dta')
unweighted_individual_balance_sheet_2006_07.drop(["seq01", "seq02", "seq03", "seq04", "seq05", "seq06", "seq07", "seq08", "seq09", "seq11", "seq12", "seq15"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2006_07.rename(index=str, inplace=True, columns={
        "seq10": "industry_sector",
        "seq13": "monthly_income", 
        "seq14": "months_worked", 
        "seq16": "other_annual_earnings"
})
unweighted_individual_balance_sheet_2006_07["hhcode"] = unweighted_individual_balance_sheet_2006_07["hhcode"].astype(int)
unweighted_individual_balance_sheet_2006_07["monthly_income"] = unweighted_individual_balance_sheet_2006_07["monthly_income"].fillna(0)
unweighted_individual_balance_sheet_2006_07["months_worked"] = unweighted_individual_balance_sheet_2006_07["months_worked"].fillna(0)
unweighted_individual_balance_sheet_2006_07["other_annual_earnings"] = unweighted_individual_balance_sheet_2006_07["other_annual_earnings"].fillna(0)
unweighted_individual_balance_sheet_2006_07["income"] = (unweighted_individual_balance_sheet_2006_07["monthly_income"] * unweighted_individual_balance_sheet_2006_07["months_worked"]) + unweighted_individual_balance_sheet_2006_07["other_annual_earnings"]
unweighted_individual_balance_sheet_2006_07.drop(["monthly_income", "months_worked", "other_annual_earnings"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2006_07 = unweighted_individual_balance_sheet_2006_07[pd.notnull(unweighted_individual_balance_sheet_2006_07["income"])]
unweighted_individual_balance_sheet_2006_07 = unweighted_individual_balance_sheet_2006_07[unweighted_individual_balance_sheet_2006_07["income"] != 0]
unweighted_individual_balance_sheet_2006_07[0:5]

,hhcode,province,district,region,psu,sec,idc,industry_sector,income
0,1001100101,punjab,islamabad,urban,10011001,E,1,construction,72000.0
6,1001100102,punjab,islamabad,urban,10011001,E,4,other,300000.0
7,1001100102,punjab,islamabad,urban,10011001,E,5,other,36000.0
8,1001100103,punjab,islamabad,urban,10011001,E,1,other,42000.0
9,1001100103,punjab,islamabad,urban,10011001,E,2,other,48000.0


In [242]:
unweighted_hh_balance_sheet_2006_07 = unweighted_individual_balance_sheet_2006_07.groupby(["hhcode", "province", "district", "region", "psu"]).agg({'income': 'sum'})
unweighted_hh_balance_sheet_2006_07.reset_index(level=unweighted_hh_balance_sheet_2006_07.index.names, inplace=True)
unweighted_hh_balance_sheet_2006_07[0:5]

,hhcode,province,district,region,psu,income
0,1001100101,punjab,islamabad,urban,10011001,72000.0
1,1001100102,punjab,islamabad,urban,10011001,336000.0
2,1001100103,punjab,islamabad,urban,10011001,120000.0
3,1001100104,punjab,islamabad,urban,10011001,204000.0
4,1001100105,punjab,islamabad,urban,10011001,108000.0


In [243]:
# HOUSEHOLD BALANCE SHEET
hh_balance_sheet_2006_07 = unweighted_hh_balance_sheet_2006_07.join(weights_2006_07.set_index('hhcode'), on='hhcode')
hh_balance_sheet_2006_07[0:5]

,hhcode,province,district,region,psu,income,weights
0,1001100101,punjab,islamabad,urban,10011001,72000.0,104.007
1,1001100102,punjab,islamabad,urban,10011001,336000.0,104.007
2,1001100103,punjab,islamabad,urban,10011001,120000.0,104.007
3,1001100104,punjab,islamabad,urban,10011001,204000.0,104.007
4,1001100105,punjab,islamabad,urban,10011001,108000.0,104.007


In [244]:
# INDIVIDUAL BALANCE SHEET
individual_characteristics_2006_07 = pd.read_stata('./heisSurvey/2006-07/section b with weights.dta')
individual_characteristics_2006_07["hhcode"] = individual_characteristics_2006_07["hhcode"].astype(int)
individual_characteristics_2006_07.drop(["section", "sbq42", "sbq43", "province", "district", "psu", "region", "sbq02", "sbq05"], inplace=True, axis=1)
individual_characteristics_2006_07.rename(index=str, inplace=True, columns={
        "sbq01": "sex", 
        "sbq03": "relation_to_head", 
        "sbq41": "birth_year"
})
individual_balance_sheet_2006_07 = unweighted_individual_balance_sheet_2006_07.join(individual_characteristics_2006_07.set_index(["hhcode", "idc"]), on=["hhcode", "idc"])
print("num_rows", individual_balance_sheet_2006_07.shape[0])
individual_balance_sheet_2006_07[0:5]

num_rows 108427


,hhcode,province,district,region,psu,sec,idc,industry_sector,income,sex,relation_to_head,birth_year,age,weights
0,1001100101,punjab,islamabad,urban,10011001,E,1,construction,72000.0,male,head of household,1983,22,104.007
6,1001100102,punjab,islamabad,urban,10011001,E,4,other,300000.0,male,son/daughter,1984,21,104.007
7,1001100102,punjab,islamabad,urban,10011001,E,5,other,36000.0,male,son/daughter,1987,18,104.007
8,1001100103,punjab,islamabad,urban,10011001,E,1,other,42000.0,male,head of household,1985,20,104.007
9,1001100103,punjab,islamabad,urban,10011001,E,2,other,48000.0,male,father/mother,1959,46,104.007


### 2007 - 2008

In [245]:
weights_2007_08 = pd.read_stata('./heisSurvey/2007-08/plist.dta')
weights_2007_08 = weights_2007_08.drop(["s1aq05a", "s1aq05b", "s1aq04", "s1aq06", "s1aq07", "s1aq08", "s1aq09", "s1aq10"], axis=1)
weights_2007_08["hhcode"] = weights_2007_08["hhcode"].astype(int)
weights_2007_08.rename(index=str, inplace=True, columns={
        "s1aq02": "relation_to_head",
        "s1aq03": "sex",
        "s1aq05c": "birth_year"
})
weights_2007_08[0:5]

,hhcode,sec,idc,relation_to_head,sex,age,birth_year,province,region,psu,weight
0,1011010101,01A,1,head,male,26,1981,punjab,urban,10110101,1207.521
1,1011010101,01A,2,brother/sister,male,23,1984,punjab,urban,10110101,1207.521
2,1011010101,01A,51,brother/sister,female,22,1985,punjab,urban,10110101,1207.521
3,1011010101,01A,52,brother/sister,female,20,1987,punjab,urban,10110101,1207.521
4,1011010101,01A,53,father/mother,female,65,1942,punjab,urban,10110101,1207.521


In [246]:
unweighted_hh_balance_sheet_2007_08 = pd.read_stata('./heisSurvey/2007-08/sec 12 c.dta')
unweighted_hh_balance_sheet_2007_08["hhcode"] = unweighted_hh_balance_sheet_2007_08["hhcode"].astype(int)
unweighted_hh_balance_sheet_2007_08.rename(index=str, inplace=True, columns={
        "s12cq01": "income", 
        "s12cq02": "expenditure", 
        "s12cq03": "ratio", 
        "s12cq04": "does_ratio_make_sense"
})
print("num_rows", unweighted_hh_balance_sheet_2007_08.shape[0])
unweighted_hh_balance_sheet_2007_08[0:5]

num_rows 15511


,hhcode,income,expenditure,ratio,does_ratio_make_sense
0,1011010101,78000,86262,0.90,yes
1,1011010102,180000,275674,0.65,no
2,1011010103,24000,81961,0.30,no
3,1011010104,144000,151283,0.95,yes
4,1011010105,1360000,557533,2.44,yes


In [247]:
# If the ratio did not make sense, a more thorough accounting involving transfers, 
# buildings, financial assets, agricultural assets etc were taken into account.
capital_2007_08 = pd.read_stata('./heisSurvey/2007-08/sec 12e.dta')
capital_2007_08["hhcode"] = capital_2007_08["hhcode"].astype(int)
capital_2007_08.rename(index=str, inplace=True, columns={
        "s12eq01": "income", 
        "s12eq02": "expenditure", 
        "s12eq03": "ratio", 
        "s12eq04": "does_ratio_make_sense"
})
print("num_rows", capital_2007_08.shape[0])
capital_2007_08[0:5]

num_rows 3085


,hhcode,sec,income,expenditure,ratio,does_ratio_make_sense
0,1011010102,12E,280000,279574,1.00,yes
1,1011010103,12E,72000,82761,0.86,yes
2,1011010109,12E,126800,107382,1.18,yes
3,1011010112,12E,40000,10700,3.74,yes
4,1011010210,12E,231500,140644,1.65,yes


In [248]:
# replace incomes as the "capital_2007_08" table is more accurate
for i in range(0, capital_2007_08.shape[0]):
    hhcode = capital_2007_08["hhcode"][i]
    new_income = capital_2007_08["income"][i]
    new_expenditure = capital_2007_08["expenditure"][i]
    new_ratio = capital_2007_08["ratio"][i]
    new_message = 'yes' if new_ratio >= 0.85 else 'no'
    
    row_index = unweighted_hh_balance_sheet_2007_08.index[unweighted_hh_balance_sheet_2007_08['hhcode'] == hhcode].tolist()[0]
    unweighted_hh_balance_sheet_2007_08.at[row_index, "income"] = new_income
    unweighted_hh_balance_sheet_2007_08.at[row_index, "expenditure"] = new_expenditure
    unweighted_hh_balance_sheet_2007_08.at[row_index, "ratio"] = new_ratio
    unweighted_hh_balance_sheet_2007_08.at[row_index, "does_ratio_make_sense"] = new_message

In [249]:
# HOUSEHOLD BALANCE SHEET
weights_2007_08_subset = weights_2007_08[["hhcode", "weight", "region", "province"]].drop_duplicates().set_index('hhcode')
hh_balance_sheet_2007_08 = unweighted_hh_balance_sheet_2007_08.join(weights_2007_08_subset, on='hhcode')
hh_balance_sheet_2007_08.rename(index=str, inplace=True, columns={
        "weight": "weights"   
})
hh_balance_sheet_2007_08[0:5]

,hhcode,income,expenditure,ratio,does_ratio_make_sense,weights,region,province
0,1011010101,78000,86262,0.90,yes,1207.521,urban,punjab
1,1011010102,280000,279574,1.00,yes,1207.521,urban,punjab
2,1011010103,72000,82761,0.86,yes,1207.521,urban,punjab
3,1011010104,144000,151283,0.95,yes,1207.521,urban,punjab
4,1011010105,1360000,557533,2.44,yes,1207.521,urban,punjab


In [250]:
# INDIVIDUAL BALANCE SHEET
unweighted_individual_balance_sheet_2007_08 = pd.read_stata('./heisSurvey/2007-08/sec 12a.dta')
unweighted_individual_balance_sheet_2007_08["hhcode"] = unweighted_individual_balance_sheet_2007_08["hhcode"].astype(int)
unweighted_individual_balance_sheet_2007_08 = unweighted_individual_balance_sheet_2007_08.drop(["s12aq01", "s12aq02", "s12aq03", "s12aq04", "s12aq05", "s12aq06", "s12aq07"], axis=1)
unweighted_individual_balance_sheet_2007_08.rename(index=str, inplace=True, columns={
        "s12aq08": "income"
})

individual_balance_sheet_2007_08 = unweighted_individual_balance_sheet_2007_08.merge(weights_2007_08, how="inner", on=["hhcode", "idc"])
individual_balance_sheet_2007_08 = individual_balance_sheet_2007_08.drop(["sec_x", "sec_y", "province_y", "region_y"], axis=1)
individual_balance_sheet_2007_08.rename(index=str, inplace=True, columns={
        "province_x": "province",
        "region_x": "region",
        "weight": "weights"
})
individual_balance_sheet_2007_08[0:5]

,hhcode,province,region,idc,income,relation_to_head,sex,age,birth_year,psu,weights
0,1011010101,punjab,urban,1,78000.0,head,male,26,1981,10110101,1207.521
1,1011010102,punjab,urban,1,180000.0,head,male,45,1962,10110101,1207.521
2,1011010103,punjab,urban,1,24000.0,head,male,50,1957,10110101,1207.521
3,1011010104,punjab,urban,1,144000.0,head,male,41,1966,10110101,1207.521
4,1011010105,punjab,urban,1,64000.0,head,male,59,1948,10110101,1207.521


In [251]:
# Note these individuals didn't have income filled in (data entry error)
individual_balance_sheet_2007_08[pd.isnull(individual_balance_sheet_2007_08["income"])][0:5]

,hhcode,province,region,idc,income,relation_to_head,sex,age,birth_year,psu,weights
11402,2012030101,sindh,rural,51,NaN,spouse,female,48,1959,20120301,1529.171


In [252]:
z = pd.read_stata('./heisSurvey/2007-08/sec1b.dta')
z.rename(index=str, inplace=True, columns={
        "s1bq05": "industry_sector",
        "s1bq08": "monthly_income", 
        "s1bq09": "months_worked", 
        "s1bq10": "annual_earnings",
        "s1bq15": "other_annual_earnings",
        "s1bq17": "other_other_annual_earnings",
        "s1bq19": "selling_wages_annual_earnings",
        "s1bq21": "pension_annual_earnings"
})
z["hhcode"] = z["hhcode"].astype(int)

# replace NaNs with 0s
z.monthly_income.fillna(0, inplace=True)
z.months_worked.fillna(0, inplace=True)
z.annual_earnings.fillna(0, inplace=True)
z.other_annual_earnings.fillna(0, inplace=True)
z.other_annual_earnings.fillna(0, inplace=True)
z.other_other_annual_earnings.fillna(0, inplace=True)
z.selling_wages_annual_earnings.fillna(0, inplace=True)
z.pension_annual_earnings.fillna(0, inplace=True)

In [253]:
for i in range(0, individual_balance_sheet_2007_08.shape[0]):
    if pd.isnull(individual_balance_sheet_2007_08["income"][i]) and individual_balance_sheet_2007_08["hhcode"][i] == 2012030101 and individual_balance_sheet_2007_08["idc"][i] == 51:
        x = z[np.logical_and(z["idc"] == idc, z["hhcode"] == hhcode)]
        # calculate income
        income = (x["monthly_income"] * x["months_worked"]) + \
            x["annual_earnings"] + \
            x["other_annual_earnings"] + \
            x["other_other_annual_earnings"] + \
            x["selling_wages_annual_earnings"] + \
            x["pension_annual_earnings"]
        individual_balance_sheet_2007_08["income"][i] = income

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [254]:
# No individuals with no income reported
individual_balance_sheet_2005_06[np.logical_and(individual_balance_sheet_2005_06["hhcode"] == 2012030101, individual_balance_sheet_2005_06["idc"] == 1)]

,hhcode,province,region,idc,income,industry_sector,relation_to_head,sex,birth_year,weights
11303,2012030101,sindh,rural,1,49200.0,52.0,head,male,1969.0,1492.364


### 2008 - 2009

In [255]:
weights_2008_09 = pd.read_stata('./heisSurvey/2008-09/weights_file.dta')
weights_2008_09[0:5]

,psu,weights
0,10011001,109.514999
1,10011002,82.794998
2,10011003,85.516998
3,10011004,80.086998
4,10011005,91.502998


In [256]:
unweighted_individual_balance_sheet_2008_09 = pd.read_stata('./heisSurvey/2008-09/section_e.dta')
unweighted_individual_balance_sheet_2008_09.drop(["seq01", "seq02", "seq03", "seq04", "seq05", "seq06", "seq07", "seq08", "seq09", "seq11", "seq12", "seq15"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2008_09.rename(index=str, inplace=True, columns={
        "seq10": "industry_sector",
        "seq13": "monthly_income", 
        "seq14": "months_worked", 
        "seq16": "other_annual_earnings"
})
unweighted_individual_balance_sheet_2008_09["hhcode"] = unweighted_individual_balance_sheet_2008_09["hhcode"].astype(int)
unweighted_individual_balance_sheet_2008_09.monthly_income = unweighted_individual_balance_sheet_2008_09.monthly_income.fillna(0)
unweighted_individual_balance_sheet_2008_09.months_worked = unweighted_individual_balance_sheet_2008_09.months_worked.fillna(0)
unweighted_individual_balance_sheet_2008_09.other_annual_earnings = unweighted_individual_balance_sheet_2008_09.other_annual_earnings.fillna(0)
unweighted_individual_balance_sheet_2008_09["income"] = (unweighted_individual_balance_sheet_2008_09["monthly_income"] * unweighted_individual_balance_sheet_2008_09["months_worked"]) + unweighted_individual_balance_sheet_2008_09["other_annual_earnings"]
unweighted_individual_balance_sheet_2008_09.drop(["monthly_income", "months_worked", "other_annual_earnings"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2008_09 = unweighted_individual_balance_sheet_2008_09[pd.notnull(unweighted_individual_balance_sheet_2008_09["income"])]
unweighted_individual_balance_sheet_2008_09 = unweighted_individual_balance_sheet_2008_09[unweighted_individual_balance_sheet_2008_09["income"] != 0]
unweighted_individual_balance_sheet_2008_09[0:5]

,hhcode,province,district,region,psu,section,idc,industry_sector,income
0,1001100101,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,175200.0
4,1001100102,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,168000.0
5,1001100102,punjab,islamabad,urban,10011001,0e1,2.0,social & personal service,138000.0
11,1001100103,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,186000.0
16,1001100104,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,216000.0


In [257]:
unweighted_hh_balance_sheet_2008_09 = unweighted_individual_balance_sheet_2008_09.groupby(["hhcode", "province", "district", "region", "psu"]).agg({'income': 'sum'})
unweighted_hh_balance_sheet_2008_09.reset_index(level=unweighted_hh_balance_sheet_2008_09.index.names, inplace=True)
unweighted_hh_balance_sheet_2008_09[0:5]

,hhcode,province,district,region,psu,income
0,1001100101,punjab,islamabad,urban,10011001,175200.0
1,1001100102,punjab,islamabad,urban,10011001,306000.0
2,1001100103,punjab,islamabad,urban,10011001,186000.0
3,1001100104,punjab,islamabad,urban,10011001,216000.0
4,1001100105,punjab,islamabad,urban,10011001,960000.0


In [258]:
# HOUSEHOLD BALANCE SHEET
hh_balance_sheet_2008_09 = unweighted_hh_balance_sheet_2008_09.join(weights_2008_09.set_index('psu'), on='psu')
hh_balance_sheet_2008_09[0:5]

,hhcode,province,district,region,psu,income,weights
0,1001100101,punjab,islamabad,urban,10011001,175200.0,109.514999
1,1001100102,punjab,islamabad,urban,10011001,306000.0,109.514999
2,1001100103,punjab,islamabad,urban,10011001,186000.0,109.514999
3,1001100104,punjab,islamabad,urban,10011001,216000.0,109.514999
4,1001100105,punjab,islamabad,urban,10011001,960000.0,109.514999


In [259]:
# INDIVIDUAL BALANCE SHEET
individual_characteristics_2008_09 = pd.read_stata('./heisSurvey/2008-09/sec_b.dta')
individual_characteristics_2008_09 = individual_characteristics_2008_09.join(weights_2008_09.set_index("psu"), on="psu")
individual_characteristics_2008_09["hhcode"] = individual_characteristics_2008_09["hhcode"].astype(int)
individual_characteristics_2008_09.drop(["sec", "sbq42", "sbq43", "province", "district", "psu", "region", "sbq02", "sbq05"], inplace=True, axis=1)
individual_characteristics_2008_09.rename(index=str, inplace=True, columns={
        "sbq01": "sex", 
        "sbq03": "relation_to_head", 
        "sbq41": "birth_year"
})
individual_balance_sheet_2008_09 = unweighted_individual_balance_sheet_2008_09.join(individual_characteristics_2008_09.set_index(["hhcode", "idc"]), on=["hhcode", "idc"])
print("num_rows", individual_balance_sheet_2008_09.shape[0])
individual_balance_sheet_2008_09[0:5]

num_rows 108698


,hhcode,province,district,region,psu,section,idc,industry_sector,income,sex,relation_to_head,birth_year,age,weights
0,1001100101,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,175200.0,male,head of household,1962.0,46.0,109.514999
4,1001100102,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,168000.0,male,head of household,1967.0,41.0,109.514999
5,1001100102,punjab,islamabad,urban,10011001,0e1,2.0,social & personal service,138000.0,female,wife/husband,1974.0,34.0,109.514999
11,1001100103,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,186000.0,male,head of household,1958.0,50.0,109.514999
16,1001100104,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,216000.0,male,head of household,1963.0,45.0,109.514999


In [260]:
# Note these individuals didn't have weights filled in (data entry error)
individual_balance_sheet_2008_09[pd.isnull(individual_balance_sheet_2008_09["weights"])]

,hhcode,province,district,region,psu,section,idc,industry_sector,income,sex,relation_to_head,birth_year,age,weights
9378,1022000308,punjab,rawalpindi,rural,10220003,0e1,7.0,other,180000.0,NaN,NaN,NaN,NaN,NaN
28393,1082000906,punjab,mianwali,rural,10820009,0e1,5.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN
98840,1231001204,punjab,vehari,urban,12310012,0e1,NaN,wholesale & retail trade,96000.0,NaN,NaN,NaN,NaN,NaN
110597,1252001902,punjab,multan,rural,12520019,0e1,NaN,transport & storage,42000.0,NaN,NaN,NaN,NaN,NaN
138979,1332004215,punjab,bahawalpur,rural,13320042,0e1,8.0,NaN,4500.0,NaN,NaN,NaN,NaN,NaN
213021,2202003305,sindh,mir pur khas,rural,22020033,0e1,NaN,"agriculture, forestry, fishing",125500.0,NaN,NaN,NaN,NaN,NaN
243401,3032001603,nwfp,lower dir,rural,30320016,0e1,1.0,construction,72000.0,NaN,NaN,NaN,NaN,NaN
271578,3122002412,nwfp,karak,rural,31220024,0e1,NaN,NaN,50000.0,NaN,NaN,NaN,NaN,NaN
339041,4172000112,balochistan,ketch/turbat,rural,41720001,0e1,NaN,wholesale & retail trade,72000.0,NaN,NaN,NaN,NaN,NaN
350314,4232001302,balochistan,musakhel,rural,42320013,0e1,6.0,construction,28000.0,NaN,NaN,NaN,NaN,NaN


In [261]:
for i in range(0, individual_balance_sheet_2008_09.shape[0]):
    if pd.isnull(individual_balance_sheet_2008_09["weights"][i]):
        psu = individual_balance_sheet_2008_09["psu"][i]
        weight = list(set(weights_2008_09[weights_2008_09["psu"] == psu]["weights"]))[0]
        individual_balance_sheet_2008_09["weights"][i] = weight

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [262]:
print(set(individual_balance_sheet_2008_09[pd.isnull(individual_balance_sheet_2008_09["weights"])]["psu"]))
individual_balance_sheet_2008_09[0:5]

set()


,hhcode,province,district,region,psu,section,idc,industry_sector,income,sex,relation_to_head,birth_year,age,weights
0,1001100101,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,175200.0,male,head of household,1962.0,46.0,109.514999
4,1001100102,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,168000.0,male,head of household,1967.0,41.0,109.514999
5,1001100102,punjab,islamabad,urban,10011001,0e1,2.0,social & personal service,138000.0,female,wife/husband,1974.0,34.0,109.514999
11,1001100103,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,186000.0,male,head of household,1958.0,50.0,109.514999
16,1001100104,punjab,islamabad,urban,10011001,0e1,1.0,social & personal service,216000.0,male,head of household,1963.0,45.0,109.514999


### 2009 - 2010

Data is missing from this year 😔

### 2010 - 2011

In [263]:
weights_2010_11 = pd.read_stata('./heisSurvey/2010-11/plist.dta')
weights_2010_11 = weights_2010_11.drop(["sbq04", "sbq52", "sbq53", "sbq06", "sbq07", "sbq08", "sbq09", "sbq10"], axis=1)
weights_2010_11["hhcode"] = weights_2010_11["hhcode"].astype(int)
weights_2010_11.rename(index=str, inplace=True, columns={
        "sbq02": "relation_to_head",
        "sbq03": "sex",
        "sbq51": "birth_year"
})
weights_2010_11[0:5]

,hhcode,region,province,psu,sec,idc,relation_to_head,sex,birth_year,age,weight
0,10011100101,urban,punjab,100111001,00B,1,head,male,1955,55,82.580482
1,10011100101,urban,punjab,100111001,00B,2,spouse,female,1959,51,82.580482
2,10011100101,urban,punjab,100111001,00B,3,son/daughter,female,1987,23,82.580482
3,10011100101,urban,punjab,100111001,00B,4,son/daughter,male,1989,20,82.580482
4,10011100101,urban,punjab,100111001,00B,5,son/daughter,male,1979,30,82.580482


In [264]:
unweighted_hh_balance_sheet_2010_11 = pd.read_stata('./heisSurvey/2010-11/balancesheet_c.dta')
unweighted_hh_balance_sheet_2010_11["hhcode"] = unweighted_hh_balance_sheet_2010_11["hhcode"].astype(int)
unweighted_hh_balance_sheet_2010_11.rename(index=str, inplace=True, columns={
        "s12cq01": "income", 
        "s12cq02": "expenditure", 
        "s12cq03": "ratio", 
        "s12cq04": "does_ratio_make_sense"
})
print("num_rows", unweighted_hh_balance_sheet_2010_11.shape[0])
unweighted_hh_balance_sheet_2010_11[0:5]

num_rows 16341


,hhcode,sec,income,expenditure,ratio,does_ratio_make_sense
0,10011100101,BSC0,408000.0,321222.0,1.27,yes
1,10011100102,BSC0,444000.0,215660.0,2.06,yes
2,10011100103,BSC0,150000.0,167008.0,0.90,yes
3,10011100104,BSC0,336000.0,165961.0,2.02,yes
4,10011100105,BSC0,192000.0,147840.0,1.30,yes


In [265]:
# If the ratio did not make sense, a more thorough accounting involving transfers, 
# buildings, financial assets, agricultural assets etc were taken into account.
capital_2010_11 = pd.read_stata('./heisSurvey/2010-11/balancesheet_e.dta')
capital_2010_11["hhcode"] = capital_2010_11["hhcode"].astype(int)
capital_2010_11.rename(index=str, inplace=True, columns={
        "s12eq01": "income", 
        "s12eq02": "expenditure", 
        "s12eq03": "ratio", 
        "s12eq04": "does_ratio_make_sense"
})
print("num_rows", capital_2010_11.shape[0])
capital_2010_11[0:5]

num_rows 3923


,hhcode,sec,income,expenditure,ratio,does_ratio_make_sense
0,10011100112,BSE,332400,346880,0.96,yes
1,10011200102,BSE,149000,140292,1.06,yes
2,10011200110,BSE,386000,389420,0.99,yes
3,10011200112,BSE,236000,243340,0.97,yes
4,10011300104,BSE,98000,103384,0.94,yes


In [266]:
# replace incomes as the "capital_2010_11" table is more accurate
for i in range(0, capital_2010_11.shape[0]):
    hhcode = capital_2010_11["hhcode"][i]
    new_income = capital_2010_11["income"][i]
    new_expenditure = capital_2010_11["expenditure"][i]
    new_ratio = capital_2010_11["ratio"][i]
    new_message = 'yes' if new_ratio >= 0.85 else 'no'
    
    row_index = unweighted_hh_balance_sheet_2010_11.index[unweighted_hh_balance_sheet_2010_11['hhcode'] == hhcode].tolist()[0]
    unweighted_hh_balance_sheet_2010_11.at[row_index, "income"] = new_income
    unweighted_hh_balance_sheet_2010_11.at[row_index, "expenditure"] = new_expenditure
    unweighted_hh_balance_sheet_2010_11.at[row_index, "ratio"] = new_ratio
    unweighted_hh_balance_sheet_2010_11.at[row_index, "does_ratio_make_sense"] = new_message

In [267]:
# HOUSEHOLD BALANCE SHEET
weights_2010_11_subset = weights_2010_11[["hhcode", "weight", "region", "province"]].drop_duplicates().set_index('hhcode')
hh_balance_sheet_2010_11 = unweighted_hh_balance_sheet_2010_11.join(weights_2010_11_subset, on='hhcode')
hh_balance_sheet_2010_11.rename(index=str, inplace=True, columns={
        "weight": "weights" 
})
hh_balance_sheet_2010_11[0:5]

,hhcode,sec,income,expenditure,ratio,does_ratio_make_sense,weights,region,province
0,10011100101,BSC0,408000.0,321222.0,1.27,yes,82.580482,urban,punjab
1,10011100102,BSC0,444000.0,215660.0,2.06,yes,82.580482,urban,punjab
2,10011100103,BSC0,150000.0,167008.0,0.90,yes,82.580482,urban,punjab
3,10011100104,BSC0,336000.0,165961.0,2.02,yes,82.580482,urban,punjab
4,10011100105,BSC0,192000.0,147840.0,1.30,yes,82.580482,urban,punjab


In [268]:
# INDIVIDUAL BALANCE SHEET
unweighted_individual_balance_sheet_2010_11 = pd.read_stata('./heisSurvey/2010-11/balancesheet_a.dta')
unweighted_individual_balance_sheet_2010_11["hhcode"] = unweighted_individual_balance_sheet_2010_11["hhcode"].astype(int)
unweighted_individual_balance_sheet_2010_11 = unweighted_individual_balance_sheet_2010_11.drop(["s12aq01", "s12aq02", "s12aq03", "s12aq04", "s12aq05", "s12aq06", "s12aq07"], axis=1)
unweighted_individual_balance_sheet_2010_11.rename(index=str, inplace=True, columns={
        "s12aq08": "income"
})

individual_balance_sheet_2010_11 = unweighted_individual_balance_sheet_2010_11.merge(weights_2010_11, how="inner", on=["hhcode", "idc"])
individual_balance_sheet_2010_11 = individual_balance_sheet_2010_11.drop(["sec_x", "sec_y", "province_y", "region_y"], axis=1)
individual_balance_sheet_2010_11.rename(index=str, inplace=True, columns={
        "province_x": "province",
        "region_x": "region",
        "weight": "weights"
})
individual_balance_sheet_2010_11[0:5]

,hhcode,province,region,idc,income,psu,relation_to_head,sex,birth_year,age,weights
0,10011100101,punjab,urban,1,180000.0,100111001,head,male,1955,55,82.580482
1,10011100101,punjab,urban,2,108000.0,100111001,spouse,female,1959,51,82.580482
2,10011100101,punjab,urban,5,120000.0,100111001,son/daughter,male,1979,30,82.580482
3,10011100102,punjab,urban,1,24000.0,100111001,head,male,1940,70,82.580482
4,10011100102,punjab,urban,3,300000.0,100111001,son/daughter,male,1980,30,82.580482


In [269]:
# Note these individuals didn't have income filled in (data entry error)
individual_balance_sheet_2010_11[pd.isnull(individual_balance_sheet_2010_11["income"])]

,hhcode,province,region,idc,income,psu,relation_to_head,sex,birth_year,age,weights
19207,31420200115,nwfp,rural,3,NaN,314202001,son/daughter,male,1995,15,1142.199951


In [270]:
z = pd.read_stata('./heisSurvey/2010-11/sec_e.dta')
z.rename(index=str, inplace=True, columns={
        "seq05": "industry_sector",
        "seq08": "monthly_income", 
        "seq09": "months_worked", 
        "seq10": "annual_earnings",
        "seq15": "other_annual_earnings",
        "seq17": "other_other_annual_earnings",
        "seq19": "selling_wages_annual_earnings",
        "seq21": "pension_annual_earnings"
})
z["hhcode"] = z["hhcode"].astype(int)

# replace NaNs with 0s
z.monthly_income.fillna(0, inplace=True)
z.months_worked.fillna(0, inplace=True)
z.annual_earnings.fillna(0, inplace=True)
z.other_annual_earnings.fillna(0, inplace=True)
z.other_annual_earnings.fillna(0, inplace=True)
z.other_other_annual_earnings.fillna(0, inplace=True)
z.selling_wages_annual_earnings.fillna(0, inplace=True)
z.pension_annual_earnings.fillna(0, inplace=True)

In [271]:
for i in range(0, individual_balance_sheet_2010_11.shape[0]):
    if individual_balance_sheet_2010_11["hhcode"][i] == 31420200115 and individual_balance_sheet_2010_11["idc"][i] == 3:
        x = z[np.logical_and(z["idc"] == 3, z["hhcode"] == 31420200115)]
        # calculate income
        income = (x["monthly_income"] * x["months_worked"]) + \
            x["annual_earnings"] + \
            x["other_annual_earnings"] + \
            x["other_other_annual_earnings"] + \
            x["selling_wages_annual_earnings"] + \
            x["pension_annual_earnings"]
        individual_balance_sheet_2010_11["income"][i] = income

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [272]:
# Note these individuals didn't have income filled in (data entry error)
individual_balance_sheet_2010_11[np.logical_and(individual_balance_sheet_2010_11["idc"] == 3, individual_balance_sheet_2010_11["hhcode"] == 31420200115)]

,hhcode,province,region,idc,income,psu,relation_to_head,sex,birth_year,age,weights
19207,31420200115,nwfp,rural,3,5000.0,314202001,son/daughter,male,1995,15,1142.199951


### 2011 - 2012

In [273]:
weights_2011_12 = pd.read_stata('./heisSurvey/2011-12/plist.dta')
weights_2011_12 = weights_2011_12.drop(["psu", "s1aq5a", "s1aq5b", "s1aq04", "s1aq06", "s1aq07", "s1aq08", "s1aq09", "s1aq10"], axis=1)
weights_2011_12["hhcode"] = weights_2011_12["hhcode"].astype(int)

weights_2011_12.rename(index=str, inplace=True, columns={
        "s1aq02": "relation_to_head",
        "s1aq03": "sex",
        "s1aq5c": "birth_year",
        "idc": "memno",
        "weight": "weights"
})
weights_2011_12[0:5]

,hhcode,province,region,sec,memno,relation_to_head,sex,age,birth_year,weights
0,1011010101,punjab,urban,01A,1,son/daughter,male,38,1973,1300.094971
1,1011010101,punjab,urban,01A,51,head,female,70,1941,1300.094971
2,1011010101,punjab,urban,01A,52,son/daughter in law,female,39,1972,1300.094971
3,1011010101,punjab,urban,01A,53,grand child,male,6,2004,1300.094971
4,1011010101,punjab,urban,01A,54,grand child,male,4,2007,1300.094971


In [274]:
unweighted_hh_balance_sheet_2011_12 = pd.read_stata('./heisSurvey/2011-12/sec_12c.dta')
unweighted_hh_balance_sheet_2011_12["hhcode"] = unweighted_hh_balance_sheet_2011_12["hhcode"].astype(int)
unweighted_hh_balance_sheet_2011_12.rename(index=str, inplace=True, columns={
        "t_icom": "income", 
        "t_exp": "expenditure", 
        "ratio_1rg": "does_ratio_make_sense"
})
print("num_rows", unweighted_hh_balance_sheet_2011_12.shape[0])
unweighted_hh_balance_sheet_2011_12[0:5]

num_rows 15807


,hhcode,province,region,psu,sec,income,expenditure,ratio,does_ratio_make_sense
0,1011010101,punjab,urban,10110101.0,12C,36000.0,101229.0,0.36,no
1,1011010102,punjab,urban,10110101.0,12C,220596.0,252980.0,0.87,yes
2,1011010103,punjab,urban,10110101.0,12C,126000.0,157260.0,0.80,no
3,1011010104,punjab,urban,10110101.0,12C,132000.0,141290.0,0.93,yes
4,1011010105,punjab,urban,10110101.0,12C,132300.0,368358.0,0.36,no


In [275]:
# If the ratio did not make sense, a more thorough accounting involving transfers, 
# buildings, financial assets, agricultural assets etc were taken into account.
capital_2011_12 = pd.read_stata('./heisSurvey/2011-12/sec_12e.dta')
capital_2011_12["hhcode"] = capital_2011_12["hhcode"].astype(int)
capital_2011_12.rename(index=str, inplace=True, columns={
        "t_income": "income", 
        "t_exp": "expenditure", 
        "ratio_1rg": "does_ratio_make_sense"
})
print("num_rows", capital_2011_12.shape[0])
capital_2011_12[0:5]

num_rows 3659


,hhcode,province,region,psu,sec,income,expenditure,ratio,does_ratio_make_sense
0,1011010101,punjab,urban,10110101,12E,91000,101529,0.90,yes
1,1011010103,punjab,urban,10110101,12E,156000,159500,0.97,yes
2,1011010105,punjab,urban,10110101,12E,332300,371358,0.89,yes
3,1011010201,punjab,urban,10110102,12E,127100,142250,0.89,yes
4,1011010211,punjab,urban,10110102,12E,244800,268134,0.91,yes


In [276]:
# replace incomes as the "capital_2010_11" table is more accurate
for i in range(0, capital_2011_12.shape[0]):
    hhcode = capital_2011_12["hhcode"][i]
    new_income = capital_2011_12["income"][i]
    new_expenditure = capital_2011_12["expenditure"][i]
    new_ratio = capital_2011_12["ratio"][i]
    new_message = 'yes' if new_ratio >= 0.85 else 'no'
    
    row_index = unweighted_hh_balance_sheet_2011_12.index[unweighted_hh_balance_sheet_2011_12['hhcode'] == hhcode].tolist()[0]
    unweighted_hh_balance_sheet_2011_12.at[row_index, "income"] = new_income
    unweighted_hh_balance_sheet_2011_12.at[row_index, "expenditure"] = new_expenditure
    unweighted_hh_balance_sheet_2011_12.at[row_index, "ratio"] = new_ratio
    unweighted_hh_balance_sheet_2011_12.at[row_index, "does_ratio_make_sense"] = new_message

In [277]:
# HOUSEHOLD BALANCE SHEET
weights_2011_12_subset = weights_2011_12[["hhcode", "weights"]].drop_duplicates().set_index('hhcode')
hh_balance_sheet_2011_12 = unweighted_hh_balance_sheet_2011_12.join(weights_2011_12_subset, on='hhcode')
hh_balance_sheet_2011_12[0:5]

,hhcode,province,region,psu,sec,income,expenditure,ratio,does_ratio_make_sense,weights
0,1011010101,punjab,urban,10110101.0,12C,91000.0,101529.0,0.90,yes,1300.094971
1,1011010102,punjab,urban,10110101.0,12C,220596.0,252980.0,0.87,yes,1300.094971
2,1011010103,punjab,urban,10110101.0,12C,156000.0,159500.0,0.97,yes,1300.094971
3,1011010104,punjab,urban,10110101.0,12C,132000.0,141290.0,0.93,yes,1300.094971
4,1011010105,punjab,urban,10110101.0,12C,332300.0,371358.0,0.89,yes,1300.094971


In [278]:
# INDIVIDUAL BALANCE SHEET
unweighted_individual_balance_sheet_2011_12 = pd.read_stata('./heisSurvey/2011-12/sec_12a.dta')
unweighted_individual_balance_sheet_2011_12["hhcode"] = unweighted_individual_balance_sheet_2011_12["hhcode"].astype(int)
unweighted_individual_balance_sheet_2011_12 = unweighted_individual_balance_sheet_2011_12.drop(["bs1qc1", "bs1qc2", "bs1qc3", "bs1qc4", "bs1qc5", "bs1qc6", "bs1qc7"], axis=1)
unweighted_individual_balance_sheet_2011_12.rename(index=str, inplace=True, columns={
        "bs1qc8": "income"
})

individual_balance_sheet_2011_12 = unweighted_individual_balance_sheet_2011_12.merge(weights_2011_12, how="inner", on=["hhcode", "memno"])
individual_balance_sheet_2011_12 = individual_balance_sheet_2011_12.drop(["sec_x", "sec_y", "province_y", "region_y"], axis=1)
individual_balance_sheet_2011_12.rename(index=str, inplace=True, columns={
        "province_x": "province",
        "region_x": "region"
})
individual_balance_sheet_2011_12[0:5]

,hhcode,province,region,psu,memno,income,relation_to_head,sex,age,birth_year,weights
0,1011010101,punjab,urban,10110101,1,36000.0,son/daughter,male,38,1973,1300.094971
1,1011010102,punjab,urban,10110101,1,220596.0,head,male,58,1953,1300.094971
2,1011010103,punjab,urban,10110101,2,90000.0,other,male,27,1984,1300.094971
3,1011010103,punjab,urban,10110101,3,36000.0,other,male,18,1993,1300.094971
4,1011010104,punjab,urban,10110101,1,132000.0,head,male,60,1951,1300.094971


### 2012 - 2013

In [279]:
weights_2012_13 = pd.read_stata('./heisSurvey/2012-13/plist_1.dta')
weights_2012_13["hhcode"] = weights_2012_13["hhcode"].astype(int)
weights_2012_13 = weights_2012_13.drop(["sec", "sbq52", "sbq53", "sbq04", "sbq06", "sbq07", "sbq08", "sbq09", "sbq10"], axis=1)
weights_2012_13.rename(index=str, inplace=True, columns={
        "sbq02": "relation_to_head",
        "sbq03": "sex",
        "sbq51": "birth_year",
        "weight": "weights"
})
weights_2012_13[0:5]

,hhcode,psu,hh,province,district,region,idc,relation_to_head,sex,birth_year,age,weights
0,1111000101,11110001,1,kpk,bannu,urban,1,head,male,1964,48,63.1731
1,1111000101,11110001,1,kpk,bannu,urban,2,spouse,female,1970,42,63.1731
2,1111000101,11110001,1,kpk,bannu,urban,3,son/daughter,female,2002,10,63.1731
3,1111000101,11110001,1,kpk,bannu,urban,4,son/daughter,female,2003,9,63.1731
4,1111000101,11110001,1,kpk,bannu,urban,5,son/daughter,female,2005,7,63.1731


This year, there is no balance sheet that adds up individual survey items. We therefore reconstruct it similar to how it was created in other years.

In [280]:
unweighted_individual_balance_sheet_2012_13 = pd.read_stata('./heisSurvey/2012-13/sec_e.dta')
unweighted_individual_balance_sheet_2012_13.drop(["sec", "seq01", "seq02", "seq03", "seq04", "seq06", "seq07", "seq11", "seq12", "seq13", "seq14", "seq16", "seq18", "seq20", "seq22"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2012_13.rename(index=str, inplace=True, columns={
        "seq05": "industry_sector",
        "seq08": "monthly_income", 
        "seq09": "months_worked", 
        "seq10": "annual_earnings",
        "seq15": "other_annual_earnings",
        "seq17": "other_other_annual_earnings",
        "seq19": "selling_wages_annual_earnings",
        "seq21": "pension_annual_earnings",
        "seq23": "remittance_within_pak",
        "seq24": "remittance_outside_pak",
        "seq25": "rent_income",
        "seq26": "other_income"
})
unweighted_individual_balance_sheet_2012_13["hhcode"] = unweighted_individual_balance_sheet_2012_13["hhcode"].astype(int)

# replace NaNs with 0s
unweighted_individual_balance_sheet_2012_13.monthly_income.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.months_worked.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.other_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.other_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.other_other_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.selling_wages_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.pension_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.remittance_within_pak.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.remittance_outside_pak.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.rent_income.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2012_13.other_income.fillna(0, inplace=True)

# calculate annual income
unweighted_individual_balance_sheet_2012_13["income"] = \
    (unweighted_individual_balance_sheet_2012_13["monthly_income"] * unweighted_individual_balance_sheet_2012_13["months_worked"]) + \
    unweighted_individual_balance_sheet_2012_13["annual_earnings"] + \
    unweighted_individual_balance_sheet_2012_13["other_annual_earnings"] + \
    unweighted_individual_balance_sheet_2012_13["other_other_annual_earnings"] + \
    unweighted_individual_balance_sheet_2012_13["selling_wages_annual_earnings"] + \
    unweighted_individual_balance_sheet_2012_13["pension_annual_earnings"] + \
    unweighted_individual_balance_sheet_2012_13["remittance_within_pak"] + \
    unweighted_individual_balance_sheet_2012_13["remittance_outside_pak"] + \
    unweighted_individual_balance_sheet_2012_13["rent_income"] + \
    unweighted_individual_balance_sheet_2012_13["other_income"]

# remove those with no income
unweighted_individual_balance_sheet_2012_13 = unweighted_individual_balance_sheet_2012_13[pd.notnull(unweighted_individual_balance_sheet_2012_13["income"])]
unweighted_individual_balance_sheet_2012_13 = unweighted_individual_balance_sheet_2012_13[unweighted_individual_balance_sheet_2012_13["income"] != 0]

unweighted_individual_balance_sheet_2012_13.drop(["annual_earnings", "monthly_income", "months_worked", "other_annual_earnings", "other_other_annual_earnings", "selling_wages_annual_earnings", "pension_annual_earnings", "remittance_within_pak", "remittance_outside_pak", "rent_income", "other_income"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2012_13[0:5]

,hhcode,province,region,district,idc,industry_sector,income
0,1111000101,1,1,111,1.0,45.0,216000.0
1,1111000101,1,1,111,2.0,85.0,300000.0
4,1111000102,1,1,111,1.0,85.0,228000.0
7,1111000103,1,1,111,1.0,47.0,500000.0
19,1111000105,1,1,111,3.0,47.0,144000.0


In [281]:
unweighted_hh_balance_sheet_2012_13 = unweighted_individual_balance_sheet_2012_13.groupby(["hhcode", "province", "district", "region"]).agg({'income': 'sum'})
unweighted_hh_balance_sheet_2012_13.reset_index(level=unweighted_hh_balance_sheet_2012_13.index.names, inplace=True)
unweighted_hh_balance_sheet_2012_13[0:5]

,hhcode,province,district,region,income
0,1111000101,1,111,1,516000.0
1,1111000102,1,111,1,228000.0
2,1111000103,1,111,1,500000.0
3,1111000105,1,111,1,144000.0
4,1111000106,1,111,1,384000.0


In [282]:
# HOUSEHOLD BALANCE SHEET
weights_2012_13_subset = weights_2012_13[["hhcode", "weights", "psu"]].drop_duplicates().set_index('hhcode')
hh_balance_sheet_2012_13 = unweighted_hh_balance_sheet_2012_13.join(weights_2012_13_subset, on='hhcode')
hh_balance_sheet_2012_13[0:5]

,hhcode,province,district,region,income,weights,psu
0,1111000101,1,111,1,516000.0,63.1731,11110001.0
1,1111000102,1,111,1,228000.0,63.1731,11110001.0
2,1111000103,1,111,1,500000.0,63.1731,11110001.0
3,1111000105,1,111,1,144000.0,63.1731,11110001.0
4,1111000106,1,111,1,384000.0,63.1731,11110001.0


In [283]:
# INDIVIDUAL BALANCE SHEET
weights_2012_13_subset = weights_2012_13.drop(["province", "district", "region"], axis=1).set_index(['hhcode', 'idc'])
individual_balance_sheet_2012_13 = unweighted_individual_balance_sheet_2012_13.join(weights_2012_13_subset, on=["hhcode", "idc"])

# # add psu to table
# roster = pd.read_stata('./heisSurvey/2012-13/roster.dta')
# roster["hhcode"] = roster["hhcode"].astype(int)
# roster = roster[["psu", "hhcode"]].drop_duplicates()
# roster = roster.set_index("hhcode")
# individual_balance_sheet_2012_13 = individual_balance_sheet_2012_13.join(roster, on="hhcode")

print("num_rows", individual_balance_sheet_2012_13.shape[0])
individual_balance_sheet_2012_13[0:5]

num_rows 117639


,hhcode,province,region,district,idc,industry_sector,income,psu,hh,relation_to_head,sex,birth_year,age,weights
0,1111000101,1,1,111,1.0,45.0,216000.0,11110001.0,1.0,head,male,1964.0,48.0,63.1731
1,1111000101,1,1,111,2.0,85.0,300000.0,11110001.0,1.0,spouse,female,1970.0,42.0,63.1731
4,1111000102,1,1,111,1.0,85.0,228000.0,11110001.0,2.0,head,male,1960.0,52.0,63.1731
7,1111000103,1,1,111,1.0,47.0,500000.0,11110001.0,3.0,head,male,1973.0,39.0,63.1731
19,1111000105,1,1,111,3.0,47.0,144000.0,11110001.0,5.0,son/daughter,male,1990.0,22.0,63.1731


In [284]:
# Note these individuals didn't have weights filled in (data entry error)
print(set(individual_balance_sheet_2012_13[pd.isnull(individual_balance_sheet_2012_13["weights"])]["hhcode"]))
individual_balance_sheet_2012_13[pd.isnull(individual_balance_sheet_2012_13["weights"])]

{2741000504, 2461001507, 1212000211, 2711001510, 2711000807}


,hhcode,province,region,district,idc,industry_sector,income,psu,hh,relation_to_head,sex,birth_year,age,weights
5566,1212000211,1,2,121,1.0,NaN,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5567,1212000211,1,2,121,2.0,49.0,60000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131564,2461001507,2,1,246,1.0,41.0,144000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131565,2461001507,2,1,246,2.0,32.0,15600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171806,2711000807,2,1,271,1.0,NaN,360000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171808,2711000807,2,1,271,3.0,49.0,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171809,2711000807,2,1,271,4.0,47.0,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171810,2711000807,2,1,271,5.0,85.0,300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172194,2711001510,2,1,271,1.0,49.0,180000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172196,2711001510,2,1,271,4.0,1.0,65000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [285]:
z = pd.read_stata('./heisSurvey/2012-13/roster.dta')
list(set(z[z["hhcode"] == 2741000504]["psu"]))[0]

27410005

In [286]:
x = pd.read_stata('./heisSurvey/2012-13/weight.dta')
list(set(x[x["psu"] == 27410005]["weight"]))[0]

217.2885

In [287]:
for i in range(0, individual_balance_sheet_2012_13.shape[0]):
    if pd.isnull(individual_balance_sheet_2012_13["psu"][i]) and individual_balance_sheet_2012_13["hhcode"][i] == 2741000504:
        individual_balance_sheet_2012_13["psu"][i] = 27410005
        individual_balance_sheet_2012_13["weights"][i] = 217.2885

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [288]:
print(set(individual_balance_sheet_2012_13[pd.isnull(individual_balance_sheet_2012_13["weights"])]["hhcode"]))
individual_balance_sheet_2012_13[pd.isnull(individual_balance_sheet_2012_13["weights"])]

{2461001507, 1212000211, 2711001510, 2711000807}


,hhcode,province,region,district,idc,industry_sector,income,psu,hh,relation_to_head,sex,birth_year,age,weights
5566,1212000211,1,2,121,1.0,NaN,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5567,1212000211,1,2,121,2.0,49.0,60000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131564,2461001507,2,1,246,1.0,41.0,144000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131565,2461001507,2,1,246,2.0,32.0,15600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171806,2711000807,2,1,271,1.0,NaN,360000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171808,2711000807,2,1,271,3.0,49.0,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171809,2711000807,2,1,271,4.0,47.0,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171810,2711000807,2,1,271,5.0,85.0,300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172194,2711001510,2,1,271,1.0,49.0,180000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172196,2711001510,2,1,271,4.0,1.0,65000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [289]:
# remove data that doesn't have a weight by making the weight zero
individual_balance_sheet_2012_13.weights.fillna(0, inplace=True)
individual_balance_sheet_2012_13[individual_balance_sheet_2012_13["weights"] == 0]

,hhcode,province,region,district,idc,industry_sector,income,psu,hh,relation_to_head,sex,birth_year,age,weights
5566,1212000211,1,2,121,1.0,NaN,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5567,1212000211,1,2,121,2.0,49.0,60000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
131564,2461001507,2,1,246,1.0,41.0,144000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
131565,2461001507,2,1,246,2.0,32.0,15600.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
171806,2711000807,2,1,271,1.0,NaN,360000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
171808,2711000807,2,1,271,3.0,49.0,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
171809,2711000807,2,1,271,4.0,47.0,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
171810,2711000807,2,1,271,5.0,85.0,300000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
172194,2711001510,2,1,271,1.0,49.0,180000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
172196,2711001510,2,1,271,4.0,1.0,65000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [290]:
print(set(individual_balance_sheet_2005_06[pd.isnull(individual_balance_sheet_2005_06["weights"])]["hhcode"]))
individual_balance_sheet_2005_06[0:5]

set()


,hhcode,province,region,idc,income,industry_sector,relation_to_head,sex,birth_year,weights
0,1011010101,punjab,urban,1,38400.0,NaN,head,male,1941.0,1031.644
1,1011010102,punjab,urban,1,144000.0,61.0,head,male,1969.0,1031.644
2,1011010103,punjab,urban,1,48000.0,62.0,head,male,1934.0,1031.644
3,1011010103,punjab,urban,2,10000.0,62.0,son/daughter,male,1984.0,1031.644
4,1011010104,punjab,urban,1,81000.0,41.0,head,male,1965.0,1031.644


### 2013 - 2014

In [291]:
weights_2013_14 = pd.read_stata('./heisSurvey/2013-14/plist.dta')
weights_2013_14.drop(["psu", "hhcode", "s1aq03", "s1aq05", "s1aq62", "s1aq63", "s1aq07", "s1aq08", "s1aq09", "s1aq10", "s1aq11"], inplace=True, axis=1)
weights_2013_14.rename(index=str, inplace=True, columns={
        "s1aq02": "relation_to_head",
        "s1aq04": "sex",
        "s1aq64": "birth_year",
        "psu_new": "psu",
        "hhcode_new": "hhcode"
})
weights_2013_14["hhcode"] = weights_2013_14["hhcode"].astype(int)

# reorder columns
cols = list(weights_2013_14.columns)
cols = cols[-3:] + cols[:-3]
weights_2013_14 = weights_2013_14[cols]

weights_2013_14[0:5]

,hhcode,stratum,psu,province,region,idc,relation_to_head,sex,age,birth_year,weights
0,1111010101,11110,11110101,kpk,rural,51,head,female,37,1976,741.67865
1,1111010101,11110,11110101,kpk,rural,52,son/daughter,female,12,2001,741.67865
2,1111010101,11110,11110101,kpk,rural,53,son/daughter,male,9,2004,741.67865
3,1111010101,11110,11110101,kpk,rural,54,son/daughter,male,7,2006,741.67865
4,1111010101,11110,11110101,kpk,rural,55,son/daughter,female,5,2008,741.67865


In [292]:
unweighted_hh_balance_sheet_2013_14 = pd.read_stata('./heisSurvey/2013-14/sec_12c.dta')
unweighted_hh_balance_sheet_2013_14.drop(["psu", "hhcode", "sec"], inplace=True, axis=1)
unweighted_hh_balance_sheet_2013_14.rename(index=str, inplace=True, columns={
        "t_income": "income", 
        "t_exp": "expenditure", 
        "ratio_lrg": "does_ratio_make_sense",
        "psu_new": "psu",
        "hhcode_new": "hhcode"
})
# removes a single row which was excluded in the new hhcodes and stratum
unweighted_hh_balance_sheet_2013_14 = unweighted_hh_balance_sheet_2013_14[pd.notnull(unweighted_hh_balance_sheet_2013_14["hhcode"])]
unweighted_hh_balance_sheet_2013_14["hhcode"] = unweighted_hh_balance_sheet_2013_14["hhcode"].astype(int)

# reorder columns
cols = list(unweighted_hh_balance_sheet_2013_14.columns)
cols = cols[-3:] + cols[:-3]
unweighted_hh_balance_sheet_2013_14 = unweighted_hh_balance_sheet_2013_14[cols]

print("num_rows", unweighted_hh_balance_sheet_2013_14.shape[0])
unweighted_hh_balance_sheet_2013_14[0:5]

num_rows 17869


,hhcode,stratum,psu,province,region,income,expenditure,ratio,does_ratio_make_sense
0,1111010101,11110.0,11110101.0,kpk,rural,0.0,176960.0,0.0000,no
1,1111010102,11110.0,11110101.0,kpk,rural,280600.0,128831.0,2.1800,yes
2,1111010103,11110.0,11110101.0,kpk,rural,36000.0,111490.0,0.3220,no
3,1111010104,11110.0,11110101.0,kpk,rural,45000.0,44778.0,1.0049,yes
4,1111010105,11110.0,11110101.0,kpk,rural,240000.0,256602.0,0.9353,yes


In [293]:
# If the ratio did not make sense, a more thorough accounting involving transfers, 
# buildings, financial assets, agricultural assets etc were taken into account.
capital_2013_14 = pd.read_stata('./heisSurvey/2013-14/sec_12e.dta')
capital_2013_14.drop(["psu", "hhcode"], inplace=True, axis=1)
capital_2013_14.rename(index=str, inplace=True, columns={
        "t_income": "income", 
        "t_exp": "expenditure", 
        "ratio_lrg1": "does_ratio_make_sense",
        "psu_new": "psu",
        "hhcode_new": "hhcode"
})
capital_2013_14["hhcode"] = capital_2013_14["hhcode"].astype(int)

# reorder columns
cols = list(capital_2013_14.columns)
cols = cols[-3:] + cols[:-3]
capital_2013_14 = capital_2013_14[cols]

print("num_rows", capital_2013_14.shape[0])
capital_2013_14[0:5]

num_rows 4419


,hhcode,stratum,psu,province,region,sec,income,expenditure,ratio,does_ratio_make_sense
0,1111010101,11110,11110101,kpk,rural,12E,197600,179520.0,1.100,1
1,1111010103,11110,11110101,kpk,rural,12E,126000,113020.0,1.110,1
2,1111010108,11110,11110101,kpk,rural,12E,219500,180155.0,1.283,1
3,1111010109,11110,11110101,kpk,rural,12E,217000,250070.0,0.870,1
4,1111010111,11110,11110101,kpk,rural,12E,232000,270694.0,0.857,1


In [294]:
# replace incomes as the "capital_2013_14" table is more accurate
for i in range(0, capital_2013_14.shape[0]):
    hhcode = capital_2013_14["hhcode"][i]
    new_income = capital_2013_14["income"][i]
    new_expenditure = capital_2013_14["expenditure"][i]
    new_ratio = capital_2013_14["ratio"][i]
    new_message = 'yes' if new_ratio >= 0.85 else 'no'
    
    row_index = unweighted_hh_balance_sheet_2013_14.index[unweighted_hh_balance_sheet_2013_14['hhcode'] == hhcode].tolist()[0]
    unweighted_hh_balance_sheet_2013_14.at[row_index, "income"] = new_income
    unweighted_hh_balance_sheet_2013_14.at[row_index, "expenditure"] = new_expenditure
    unweighted_hh_balance_sheet_2013_14.at[row_index, "ratio"] = new_ratio
    unweighted_hh_balance_sheet_2013_14.at[row_index, "does_ratio_make_sense"] = new_message

In [295]:
# HOUSEHOLD BALANCE SHEET
weights_2013_14_subset = weights_2013_14[["hhcode", "weights"]].drop_duplicates().set_index('hhcode')
hh_balance_sheet_2013_14 = unweighted_hh_balance_sheet_2013_14.join(weights_2013_14_subset, on='hhcode')
hh_balance_sheet_2013_14[0:5]

,hhcode,stratum,psu,province,region,income,expenditure,ratio,does_ratio_make_sense,weights
0,1111010101,11110.0,11110101.0,kpk,rural,197600.0,179520.0,1.1000,yes,741.67865
1,1111010102,11110.0,11110101.0,kpk,rural,280600.0,128831.0,2.1800,yes,741.67865
2,1111010103,11110.0,11110101.0,kpk,rural,126000.0,113020.0,1.1100,yes,741.67865
3,1111010104,11110.0,11110101.0,kpk,rural,45000.0,44778.0,1.0049,yes,741.67865
4,1111010105,11110.0,11110101.0,kpk,rural,240000.0,256602.0,0.9353,yes,741.67865


In [296]:
# INDIVIDUAL BALANCE SHEET
unweighted_individual_balance_sheet_2013_14 = pd.read_stata('./heisSurvey/2013-14/sec_12a.dta')
unweighted_individual_balance_sheet_2013_14 = unweighted_individual_balance_sheet_2013_14.drop(["psu_new", "hhcode", "psu", "sec", "bs1qc1", "bs1qc2", "bs1qc3", "bs1qc4", "bs1qc5", "bs1qc6", "bs1qc7"], axis=1)
unweighted_individual_balance_sheet_2013_14.rename(index=str, inplace=True, columns={
        "bs1qc8": "income",
        "hhcode_new": "hhcode"
})
unweighted_individual_balance_sheet_2013_14["hhcode"] = unweighted_individual_balance_sheet_2013_14["hhcode"].astype(int)

# reorder columns
cols = list(unweighted_individual_balance_sheet_2013_14.columns)
cols = cols[-3:] + cols[:-3]
unweighted_individual_balance_sheet_2013_14 = unweighted_individual_balance_sheet_2013_14[cols]

individual_balance_sheet_2013_14 = unweighted_individual_balance_sheet_2013_14.merge(weights_2013_14, how="inner", on=["hhcode", "idc"])
individual_balance_sheet_2013_14 = individual_balance_sheet_2013_14.drop(["province_y", "region_y", "stratum_y"], axis=1)
individual_balance_sheet_2013_14.rename(index=str, inplace=True, columns={
        "province_x": "province",
        "region_x": "region",
        "stratum_x": "stratum"
})

# reorder columns
cols = list(individual_balance_sheet_2013_14.columns)
cols = cols[1:] + cols[:1]
individual_balance_sheet_2013_14 = individual_balance_sheet_2013_14[cols]

individual_balance_sheet_2013_14[0:5]

,hhcode,stratum,province,region,idc,psu,relation_to_head,sex,age,birth_year,weights,income
0,1111010102,11110,kp,rural,1.0,11110101,head,male,42,1971,741.67865,280600.0
1,1111010103,11110,kp,rural,51.0,11110101,head,female,30,1983,741.67865,36000.0
2,1111010104,11110,kp,rural,51.0,11110101,head,female,50,1963,741.67865,45000.0
3,1111010105,11110,kp,rural,1.0,11110101,head,male,54,1959,741.67865,240000.0
4,1111010106,11110,kp,rural,1.0,11110101,head,male,36,1977,741.67865,260000.0


In [297]:
# Note these individuals didn't have income filled in (data entry error)
individual_balance_sheet_2013_14[pd.isnull(individual_balance_sheet_2013_14["income"])][0:5]

,hhcode,stratum,province,region,idc,psu,relation_to_head,sex,age,birth_year,weights,income
11375,2512240901,25122,punjab,urban,1.0,25122409,head,male,63,1950,1789.033813,NaN


In [298]:
z = pd.read_stata('./heisSurvey/2013-14/sec_1b.dta', convert_categoricals=False)
z.rename(index=str, inplace=True, columns={
        "s1bq05": "industry_sector",
        "s1bq08": "monthly_income", 
        "s1bq09": "months_worked", 
        "s1bq10": "annual_earnings",
        "s1bq15": "other_annual_earnings",
        "s1bq17": "other_other_annual_earnings",
        "s1bq19": "selling_wages_annual_earnings",
        "s1bq21": "pension_annual_earnings"
})
z["hhcode_new"] = z["hhcode_new"].astype(int)

# replace NaNs with 0s
z.monthly_income.fillna(0, inplace=True)
z.months_worked.fillna(0, inplace=True)
z.annual_earnings.fillna(0, inplace=True)
z.other_annual_earnings.fillna(0, inplace=True)
z.other_annual_earnings.fillna(0, inplace=True)
z.other_other_annual_earnings.fillna(0, inplace=True)
z.selling_wages_annual_earnings.fillna(0, inplace=True)
z.pension_annual_earnings.fillna(0, inplace=True)

In [299]:
for i in range(0, individual_balance_sheet_2013_14.shape[0]):
    if pd.isnull(individual_balance_sheet_2013_14["income"][i]) and individual_balance_sheet_2013_14["hhcode"][i] == 2512240901 and individual_balance_sheet_2013_14["idc"][i] == 1.0:
        x = z[np.logical_and(z["idc"] == 1.0, z["hhcode_new"] == 2512240901)]
        # calculate income
        income = (x["monthly_income"] * x["months_worked"]) + \
            x["annual_earnings"] + \
            x["other_annual_earnings"] + \
            x["other_other_annual_earnings"] + \
            x["selling_wages_annual_earnings"] + \
            x["pension_annual_earnings"]
        individual_balance_sheet_2013_14["income"][i] = income

In [300]:
individual_balance_sheet_2013_14[np.logical_and(individual_balance_sheet_2013_14["hhcode"] == 2512240901, individual_balance_sheet_2013_14["idc"] == 1)]

,hhcode,stratum,province,region,idc,psu,relation_to_head,sex,age,birth_year,weights,income
11375,2512240901,25122,punjab,urban,1.0,25122409,head,male,63,1950,1789.033813,0.0


### 2014 - 2015

In [301]:
weights_2014_15 = pd.read_stata('./heisSurvey/2014-15/plist.dta')
weights_2014_15["hhcode"] = weights_2014_15["hhcode"].astype(int)
weights_2014_15 = weights_2014_15.drop(["sec", "psu", "sbq62", "sbq63", "sbq03", "sbq11", "sbq07", "sbq08", "sbq09", "sbq10", "sbq05"], axis=1)
weights_2014_15.rename(index=str, inplace=True, columns={
        "sbq02": "relation_to_head",
        "sbq04": "sex",
        "sbq61": "birth_year",
        "weight": "weights"
})
weights_2014_15[0:5]

,hhcode,province,region,district,idc,relation_to_head,sex,birth_year,age,weights
0,1111000101,kp,rural,chitral,1,head,male,1933,81,168.0
1,1111000101,kp,rural,chitral,2,spouse,female,1944,70,168.0
2,1111000101,kp,rural,chitral,3,son/daughter,male,1970,44,168.0
3,1111000101,kp,rural,chitral,4,son/daughter-in-law,female,1977,37,168.0
4,1111000101,kp,rural,chitral,5,grand child,male,2002,12,168.0


This year, there is no balance sheet that adds up individual survey items. We therefore reconstruct it similar to how it was created in other years.

In [302]:
unweighted_individual_balance_sheet_2014_15 = pd.read_stata('./heisSurvey/2014-15/sec_e.dta', convert_categoricals=False)
unweighted_individual_balance_sheet_2014_15.drop(["sec", "seq01", "seq02", "seq03", "seq04", "seq06", "seq07", "seq11", "seq12", "seq13", "seq14", "seq16", "seq18", "seq20", "seq22"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2014_15.rename(index=str, inplace=True, columns={
        "seq05": "industry_sector",
        "seq08": "monthly_income", 
        "seq09": "months_worked", 
        "seq10": "annual_earnings",
        "seq15": "other_annual_earnings",
        "seq17": "other_other_annual_earnings",
        "seq19": "selling_wages_annual_earnings",
        "seq21": "pension_annual_earnings",
        "seq23": "remittance_within_pak",
        "seq24": "remittance_outside_pak",
        "seq25": "rent_income",
        "seq26": "other_income"
})
unweighted_individual_balance_sheet_2014_15["hhcode"] = unweighted_individual_balance_sheet_2014_15["hhcode"].astype(int)

# replace NaNs with 0s
unweighted_individual_balance_sheet_2014_15.monthly_income.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.months_worked.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.other_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.other_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.other_other_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.selling_wages_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.pension_annual_earnings.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.remittance_within_pak.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.remittance_outside_pak.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.rent_income.fillna(0, inplace=True)
unweighted_individual_balance_sheet_2014_15.other_income.fillna(0, inplace=True)

# calculate annual income
unweighted_individual_balance_sheet_2014_15["income"] = \
    (unweighted_individual_balance_sheet_2014_15["monthly_income"] * unweighted_individual_balance_sheet_2014_15["months_worked"]) + \
    unweighted_individual_balance_sheet_2014_15["annual_earnings"] + \
    unweighted_individual_balance_sheet_2014_15["other_annual_earnings"] + \
    unweighted_individual_balance_sheet_2014_15["other_other_annual_earnings"] + \
    unweighted_individual_balance_sheet_2014_15["selling_wages_annual_earnings"] + \
    unweighted_individual_balance_sheet_2014_15["pension_annual_earnings"] + \
    unweighted_individual_balance_sheet_2014_15["remittance_within_pak"] + \
    unweighted_individual_balance_sheet_2014_15["remittance_outside_pak"] + \
    unweighted_individual_balance_sheet_2014_15["rent_income"] + \
    unweighted_individual_balance_sheet_2014_15["other_income"]

# remove those with no income
unweighted_individual_balance_sheet_2014_15 = unweighted_individual_balance_sheet_2014_15[pd.notnull(unweighted_individual_balance_sheet_2014_15["income"])]
unweighted_individual_balance_sheet_2014_15 = unweighted_individual_balance_sheet_2014_15[unweighted_individual_balance_sheet_2014_15["income"] != 0]

unweighted_individual_balance_sheet_2014_15.drop(["annual_earnings", "monthly_income", "months_worked", "other_annual_earnings", "other_other_annual_earnings", "selling_wages_annual_earnings", "pension_annual_earnings", "remittance_within_pak", "remittance_outside_pak", "rent_income", "other_income"], inplace=True, axis=1)
unweighted_individual_balance_sheet_2014_15[0:5]

,hhcode,psu,province,region,district,idc,hhno,industry_sector,income
0,1111000101,11110001,1,1,111,1,01,150.0,30000.0
2,1111000101,11110001,1,1,111,3,01,150.0,60000.0
5,1111000102,11110001,1,1,111,1,02,150.0,206000.0
9,1111000103,11110001,1,1,111,1,03,150.0,80000.0
11,1111000103,11110001,1,1,111,3,03,4100.0,90000.0


In [303]:
unweighted_hh_balance_sheet_2014_15 = unweighted_individual_balance_sheet_2014_15.groupby(["hhcode", "province", "district", "region"]).agg({'income': 'sum'})
unweighted_hh_balance_sheet_2014_15.reset_index(level=unweighted_hh_balance_sheet_2014_15.index.names, inplace=True)
unweighted_hh_balance_sheet_2014_15[0:5]

,hhcode,province,district,region,income
0,1111000101,1,111,1,90000.0
1,1111000102,1,111,1,206000.0
2,1111000103,1,111,1,170000.0
3,1111000104,1,111,1,198000.0
4,1111000105,1,111,1,160000.0


In [304]:
# HOUSEHOLD BALANCE SHEET
weights_2014_15_subset = weights_2014_15[["hhcode", "weights"]].drop_duplicates().set_index('hhcode')
hh_balance_sheet_2014_15 = unweighted_hh_balance_sheet_2014_15.join(weights_2014_15_subset, on='hhcode')
print("num_rows", hh_balance_sheet_2014_15.shape[0])
hh_balance_sheet_2014_15[0:5]

num_rows 78505


,hhcode,province,district,region,income,weights
0,1111000101,1,111,1,90000.0,168.0
1,1111000102,1,111,1,206000.0,168.0
2,1111000103,1,111,1,170000.0,168.0
3,1111000104,1,111,1,198000.0,168.0
4,1111000105,1,111,1,160000.0,168.0


In [305]:
# INDIVIDUAL BALANCE SHEET
weights_2014_15_subset = weights_2014_15.drop(["province", "district", "region"], axis=1).set_index(['hhcode', 'idc'])
individual_balance_sheet_2014_15 = unweighted_individual_balance_sheet_2014_15.join(weights_2014_15_subset, on=["hhcode", "idc"])
print("num_rows", individual_balance_sheet_2014_15.shape[0])
individual_balance_sheet_2014_15[0:5]

num_rows 122664


,hhcode,psu,province,region,district,idc,hhno,industry_sector,income,relation_to_head,sex,birth_year,age,weights
0,1111000101,11110001,1,1,111,1,01,150.0,30000.0,head,male,1933,81,168.0
2,1111000101,11110001,1,1,111,3,01,150.0,60000.0,son/daughter,male,1970,44,168.0
5,1111000102,11110001,1,1,111,1,02,150.0,206000.0,head,male,1950,64,168.0
9,1111000103,11110001,1,1,111,1,03,150.0,80000.0,head,male,1968,46,168.0
11,1111000103,11110001,1,1,111,3,03,4100.0,90000.0,son/daughter,male,1997,17,168.0


### 2015 - 2016

The final year 🙏

In [306]:
weights_2015_16 = pd.read_stata('./heisSurvey/2015-16/plist.dta')
weights_2015_16.drop(["psu", "s1aq03", "s1aq05", "s1aq61", "s1aq62", "s1aq07", "s1aq08", "s1aq09", "s1aq10", "s1aq11"], inplace=True, axis=1)
weights_2015_16.rename(index=str, inplace=True, columns={
        "s1aq02": "relation_to_head",
        "s1aq04": "sex",
        "s1aq63": "birth_year"
})
weights_2015_16["hhcode"] = weights_2015_16["hhcode"].astype(int)
weights_2015_16[0:5]

,hhcode,province,region,sec,idc,relation_to_head,sex,age,birth_year,weights
0,1102010101,kp,urban,01A,1,head,male,45,1970,89.760666
1,1102010101,kp,urban,01A,2,nephew\niece,male,22,1993,89.760666
2,1102010101,kp,urban,01A,3,nephew\niece,male,15,2000,89.760666
3,1102010101,kp,urban,01A,51,spouse,female,40,1975,89.760666
4,1102010101,kp,urban,01A,52,son/daughter,female,6,2009,89.760666


In [307]:
unweighted_hh_balance_sheet_2015_16 = pd.read_stata('./heisSurvey/2015-16/sec_9c.dta')
unweighted_hh_balance_sheet_2015_16.drop(["sec"], inplace=True, axis=1)
unweighted_hh_balance_sheet_2015_16.rename(index=str, inplace=True, columns={
        "bs3c01": "income", 
        "bs3c02": "expenditure", 
        "bs3c03": "ratio",
        "bs3c04": "does_ratio_make_sense"
})
unweighted_hh_balance_sheet_2015_16["hhcode"] = unweighted_hh_balance_sheet_2015_16["hhcode"].astype(int)

print("num_rows", unweighted_hh_balance_sheet_2015_16.shape[0])
unweighted_hh_balance_sheet_2015_16[0:5]

num_rows 24238


,hhcode,province,region,psu,income,expenditure,ratio,does_ratio_make_sense
0,1102010101,kp,urban,11020101,480000.0,208692,2.30,yes
1,1102010102,kp,urban,11020101,294000.0,160948,1.83,yes
2,1102010103,kp,urban,11020101,300520.0,186756,1.61,yes
3,1102010104,kp,urban,11020101,12900.0,284874,0.05,no
4,1102010105,kp,urban,11020101,516000.0,412734,1.30,yes


In [308]:
# If the ratio did not make sense, a more thorough accounting involving transfers, 
# buildings, financial assets, agricultural assets etc were taken into account.
capital_2015_16 = pd.read_stata('./heisSurvey/2015-16/sec_9e.dta')
capital_2015_16.rename(index=str, inplace=True, columns={
        "bs5ec01": "income", 
        "bs5ec02": "expenditure", 
        "bs5ec03": "ratio",
        "bs5ec04": "does_ratio_make_sense"
})
capital_2015_16["hhcode"] = capital_2015_16["hhcode"].astype(int)

print("num_rows", capital_2015_16.shape[0])
capital_2015_16[0:5]

num_rows 5082


,hhcode,province,region,psu,sec,filler,income,expenditure,ratio,does_ratio_make_sense
0,1102010104,kp,urban,11020101,09E,0.0,262900.0,294234,0.8900,yes
1,1102010107,kp,urban,11020101,09E,0.0,360000.0,336217,1.0700,yes
2,1102010108,kp,urban,11020101,09E,0.0,420000.0,190780,2.2014,yes
3,1102010109,kp,urban,11020101,09E,0.0,2650000.0,3129340,0.8500,yes
4,1102010111,kp,urban,11020101,09E,0.0,540000.0,505740,1.0677,yes


In [309]:
# replace incomes as the "capital_2015_16" table is more accurate
for i in range(0, capital_2015_16.shape[0]):
    hhcode = capital_2015_16["hhcode"][i]
    new_income = capital_2015_16["income"][i]
    new_expenditure = capital_2015_16["expenditure"][i]
    new_ratio = capital_2015_16["ratio"][i]
    new_message = 'yes' if new_ratio >= 0.85 else 'no'
    
    row_index = unweighted_hh_balance_sheet_2015_16.index[unweighted_hh_balance_sheet_2015_16['hhcode'] == hhcode].tolist()[0]
    unweighted_hh_balance_sheet_2015_16.at[row_index, "income"] = new_income
    unweighted_hh_balance_sheet_2015_16.at[row_index, "expenditure"] = new_expenditure
    unweighted_hh_balance_sheet_2015_16.at[row_index, "ratio"] = new_ratio
    unweighted_hh_balance_sheet_2015_16.at[row_index, "does_ratio_make_sense"] = new_message

In [310]:
# HOUSEHOLD BALANCE SHEET
weights_2015_16_subset = weights_2015_16[["hhcode", "weights"]].drop_duplicates().set_index('hhcode')
hh_balance_sheet_2015_16 = unweighted_hh_balance_sheet_2015_16.join(weights_2015_16_subset, on='hhcode')
hh_balance_sheet_2015_16[0:5]

,hhcode,province,region,psu,income,expenditure,ratio,does_ratio_make_sense,weights
0,1102010101,kp,urban,11020101,480000.0,208692,2.30,yes,89.760666
1,1102010102,kp,urban,11020101,294000.0,160948,1.83,yes,89.760666
2,1102010103,kp,urban,11020101,300520.0,186756,1.61,yes,89.760666
3,1102010104,kp,urban,11020101,262900.0,294234,0.89,yes,89.760666
4,1102010105,kp,urban,11020101,516000.0,412734,1.30,yes,89.760666


In [311]:
# INDIVIDUAL BALANCE SHEET
unweighted_individual_balance_sheet_2015_16 = pd.read_stata('./heisSurvey/2015-16/sec_9a.dta')
unweighted_individual_balance_sheet_2015_16 = unweighted_individual_balance_sheet_2015_16.drop(["sec", "bs1qc1", "bs1qc2", "bs1qc3", "bs1qc4", "bs1qc5", "bs1qc6", "bs1qc7"], axis=1)
unweighted_individual_balance_sheet_2015_16.rename(index=str, inplace=True, columns={
        "bs1qc8": "income"
})
unweighted_individual_balance_sheet_2015_16["hhcode"] = unweighted_individual_balance_sheet_2015_16["hhcode"].astype(int)

individual_balance_sheet_2015_16 = unweighted_individual_balance_sheet_2015_16.merge(weights_2015_16, how="inner", on=["hhcode", "idc"])
individual_balance_sheet_2015_16 = individual_balance_sheet_2015_16.drop(["province_y", "region_y"], axis=1)
individual_balance_sheet_2015_16.rename(index=str, inplace=True, columns={
        "province_x": "province",
        "region_x": "region"
})

individual_balance_sheet_2015_16[0:5]

,hhcode,province,region,psu,idc,income,sec,relation_to_head,sex,age,birth_year,weights
0,1102010101,kp,urban,11020101,1,420000.0,01A,head,male,45,1970,89.760666
1,1102010101,kp,urban,11020101,2,60000.0,01A,nephew\niece,male,22,1993,89.760666
2,1102010102,kp,urban,11020101,1,186000.0,01A,head,male,29,1986,89.760666
3,1102010102,kp,urban,11020101,2,108000.0,01A,father/mother,male,65,1950,89.760666
4,1102010103,kp,urban,11020101,1,180520.0,01A,head,male,67,1948,89.760666


## Tax Data

We load tax data from years 2013 - 2017. Note that the tax year starts from June. The names of the relevant tables that we create after running the code below are listed below. See the bottom of this section for code to download these tables.

The original tax PDF files are too large. Here, we download the files locally (if they have already been downloaded with the appropriate names, then we will skip over those files). WARNING: this takes awhile (each PDF is about 20,000 pages or approximately 10 minutes).

In [312]:
# Download Tax Data - takes awhile
folder_name = "taxpayerData"
tax_links = {
    "2013_ParliamentarianTax": "http://www.fbr.gov.pk/Downloads/Document/3769/1",
    "2014_ParliamentarianTax": "http://www.fbr.gov.pk/Downloads/Document/4122/1",
    "2015_ParliamentarianTax": "http://www.fbr.gov.pk/Downloads/Document/14688/1",
    "2016_ParliamentarianTax": "http://www.fbr.gov.pk/Downloads/Document/24994/1",
    "2017_ParliamentarianTax": "http://www.fbr.gov.pk/Downloads/Document/25724/1",
    "2013_Tax": "http://www.fbr.gov.pk/Downloads/Document/3808/1",
    "2014_Tax": "http://www.fbr.gov.pk/Downloads/Document/4124/1",
    "2015_Tax": "http://www.fbr.gov.pk/Downloads/Document/14689/1",
    "2016_Tax": "http://www.fbr.gov.pk/Downloads/Document/25006/1",
    "2017_Tax": "http://www.fbr.gov.pk/Downloads/Document/25725/1"
}

if not os.path.exists("./" + folder_name + "/"):
    os.makedirs("./" + folder_name + "/")
    
for k in tax_links.keys():
    if os.path.exists("./" + folder_name + "/" + k + ".pdf"):
        print("file already exists:", k)
        continue
    link = tax_links[k]
    response = urllib.request.urlopen(str(link))
    with open("./" + folder_name + "/" + k + ".pdf", 'wb') as f:
        f.write(response.read())
    print("file uploaded:", k)

file already exists: 2015_Tax
file already exists: 2017_ParliamentarianTax
file already exists: 2013_ParliamentarianTax
file already exists: 2013_Tax
file already exists: 2016_ParliamentarianTax
file already exists: 2014_ParliamentarianTax
file already exists: 2017_Tax
file already exists: 2014_Tax
file already exists: 2016_Tax
file already exists: 2015_ParliamentarianTax


### 2012 - 2013

### 2013 - 2014

### 2014 - 2015

### 2015 - 2016

### 2016 - 2017

# Combining the Data

Here we combine everything

# Visualization and Conclusions

**NOTE**: DOING THIS JUST WITH SURVEY DATA RIGHT NOW

**Helper Functions**

In [403]:
def get_sorted_table(table_type, start_year):  
    """
    Sorted by income.
    
    Args:
        str (table_type): accepts only 'hh' for household or 'individual' for individual.
        int (start_year): currently limited to 2004 - 2015.

    Returns:
        pd.DataFrame: dataframe corresponding to table.
    """
    t = eval(table_type + "_balance_sheet_" + str(start_year) + "_" + str(start_year+1)[-2:])
    return t.sort_values(by=['income'])
    
def weighted_sum(data, weights):
    """
    Args:
        pd.Series (data)
        pd.Series (weights)

    Returns:
        float: weighted sum.
    """
    cum_sum = 0
    for i in range(0, len(weights)):
        cum_sum += data[i] * weights[i]
    return cum_sum

def top_percentiles(p, sorted_data, sorted_weights):
    """
    Args:
        int (p): 0 < p < 1.
        pd.Series (data)
        pd.Series (weights)

    Returns:
        (list, list): corresponding data, and weights.
    """
    q = (1 - p) * sum(sorted_weights)
    index = len(sorted_weights)
    cum_sum = sum(sorted_weights)
    while cum_sum > q:
        index -= 1
        cum_sum -= sorted_weights[index]
    return (sorted_data[index:], sorted_weights[index:])

def bottom_percentiles(p, sorted_data, sorted_weights):
    """
    Args:
        int (p): 0 <= p <=1.
        pd.Series (data)
        pd.Series (weights)

    Returns:
        (list, list): corresponding data, and weights.
    """
    q = p * sum(sorted_weights)
    index = -1
    cum_sum = 0
    while cum_sum < q:
        index += 1
        cum_sum += sorted_weights[index]
    return (sorted_data[:index], sorted_weights[:index])

## General Basics

**Helper Functions**

## National Income Shares

**Helper Functions**

In [417]:
def calculate_income_share(table_type, year, p, top=True):
    """
    Args:
        str (table_type): accepts only 'hh' for household or 'individual' for individual.
        int (year): currently limited to 2004 - 2015.
        float (percentile)
        bool (top): getting top [percentile]% or bottom [percentile]%? defaults to top.
        
    Returns:
        float: income share.
    """
    if year == 2009:
        print("No data for 2009-2010")
        return
    t = get_table(table_type, year)
    if top:
        p_data, p_weights = top_percentiles(p, t["income"], t["weights"])
    else:
        p_data, p_weights = bottom_percentiles(p, t["income"], t["weights"])
    partial_sum = weighted_sum(p_data, p_weights)
    total_sum = weighted_sum(t["income"], t["weights"])
    return round(partial_sum / total_sum, 2)

In [ ]:
# NEED TO THINK THROUGH HOW TO COMBINE THE TAX DATA AS IT'S HALFWAY JUNE TO JUNE. ALSO HOW TO DEAL WITH WEIGHTS.

### Top 1% Income Share

In [421]:
# top 1 percent
years = np.arange(2004, 2016, 1)
index = np.argwhere(years==2009)
years = np.delete(years, index)

top_1_income_shares = []
for y in years:
    top_1_income_shares.append(calculate_income_share("individual", y, 0.01))

In [422]:
top_1_trace = go.Scatter(
    x = years,
    y = top_1_income_shares,
    name = 'Top 1%'
)

layout = dict(title = 'Top 1% National Income Share in Pakistan',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'National Income Share'),
              )
data = [top_1_trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')

### Top 0.1% Income Share

In [423]:
# top 0.1 percent
years = np.arange(2004, 2016, 1)
index = np.argwhere(years==2009)
years = np.delete(years, index)

top_01_income_shares = []
for y in years:
    top_01_income_shares.append(calculate_income_share("individual", y, 0.001))

In [424]:
top_01_trace = go.Scatter(
    x = years,
    y = top_01_income_shares,
    name = "Top 0.1%"
)

layout = dict(title = 'Top 0.1% National Income Share in Pakistan',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'National Income Share'),
              )
data = [top_01_trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')

### Top 0.01% Income Share

In [425]:
# top 0.01 percent
years = np.arange(2004, 2016, 1)
index = np.argwhere(years==2009)
years = np.delete(years, index)

top_001_income_shares = []
for y in years:
    top_001_income_shares.append(calculate_income_share("individual", y, 0.0001))

In [426]:
top_001_trace = go.Scatter(
    x = years,
    y = top_001_income_shares,
    name = "Top 0.01%"
)

layout = dict(title = 'Top 0.01% National Income Share in Pakistan',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'National Income Share'),
              )
data = [top_001_trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')

### Top 1% and Top 0.1% and Top 0.01% Income Shares

In [427]:
layout = dict(title = 'Top 1%, Top 0.1% and Top 0.01% National Income Shares in Pakistan',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'National Income Share'),
              )
data = [top_1_trace, top_01_trace, top_001_trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')

### Top 1% and Bottom 50% Income Shares

In [428]:
# bottom 50 percent
years = np.arange(2004, 2016, 1)
index = np.argwhere(years==2009)
years = np.delete(years, index)

bottom_50_income_shares = []
for y in years:
    bottom_50_income_shares.append(calculate_income_share("individual", y, 0.5, top=False))

In [429]:
bottom_50_trace = go.Scatter(
    x = years,
    y = bottom_50_income_shares,
    name = 'Bottom 50%'
)

layout = dict(title = 'Top 1% and Bottom 50% National Income Shares in Pakistan',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'National Income Share'),
              )
data = [top_1_trace, bottom_50_trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')

### Top 10% and Middle 40% Income Shares

In [430]:
# top 10 percent
years = np.arange(2004, 2016, 1)
index = np.argwhere(years==2009)
years = np.delete(years, index)

top_10_income_shares = []
for y in years:
    top_10_income_shares.append(calculate_income_share("individual", y, 0.1))

In [431]:
# middle 40 percent
years = np.arange(2004, 2016, 1)
index = np.argwhere(years==2009)
years = np.delete(years, index)

middle_40_income_shares = []
for y in years:
    top20 = calculate_income_share("individual", y, 0.3)
    bottom20 = calculate_income_share("individual", y, 0.3, top=False)
    middle_40_income_shares.append(1 - (top20 + bottom20))

In [432]:
top_10_trace = go.Scatter(
    x = years,
    y = top_10_income_shares,
    name = "top 10%"
)

middle_40_trace = go.Scatter(
    x = years,
    y = middle_40_income_shares,
    name = "middle 40%"
)

layout = dict(title = 'Top 10% and Middle 40% National Income Shares in Pakistan',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'National Income Share'),
              )
data = [top_10_trace, middle_40_trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')

## Income Growth

**Helper Functions**

## Miscellaneous

**Helper Functions**

In [ ]:
income_data = [int(x) for x in heisSection9A_IncomeNoCapital["total_income_x"] if not pd.isnull(x)]
weights = [int(x) for x in heisSection9A_IncomeNoCapital["weights"]]
weighted_income_data = []
for i in range(0, len(income_data)):
    income = [income_data[i]]
    w = weights[i]
    weighted_income_data.extend(income * w)

s_data = np.array(sorted(weighted_income_data))
n = len(s_data)
percentiles = 100 * (np.arange(1, n+1) - 0.5) / n

In [ ]:
heisSectionPList_Weights = pd.read_stata('./heisSurvey/2015-16/plist.dta')
heisSectionPList_Weights["hhcode"] = heisSectionPList_Weights["hhcode"].astype('int64')
heisSectionPList_Weights = heisSectionPList_Weights[["hhcode", "weights"]].drop_duplicates()

In [ ]:
heisSection9E_IncomeWithCapital = pd.read_excel("./heisSurvey/HEISBalanceSheet.xls")
heisSection9E_IncomeWithCapital = heisSection9E_IncomeWithCapital.rename(index=str, columns={
                                "bs5ec01": "total_income_xy", 
                                "bs5ec02": "total_expenditure_ze", 
                                "bs5ec03": "ratio", 
                                "bs5ec04": "greater_than_0.85"})
heisSection9E_IncomeWithCapital = heisSection9E_IncomeWithCapital.join(heisSectionPList_Weights.set_index('hhcode'), on='hhcode')
heisSection9E_IncomeWithCapital[0:5]

In [107]:
heisSection9A_IncomeNoCapital = pd.read_excel("./heisSurvey/HEISBalanceSheet_NoCapital.xls")
heisSection9A_IncomeNoCapital = heisSection9A_IncomeNoCapital.rename(index=str, columns={
                                "bs3c01": "total_income_x", 
                                "bs3c02": "total_expenditure_z", 
                                "bs3c03": "ratio", 
                                "bs3c04": "greater_than_0.85"})
heisSection9A_IncomeNoCapital = heisSection9A_IncomeNoCapital.join(heisSectionPList_Weights.set_index('hhcode'), on='hhcode')
heisSection9A_IncomeNoCapital[0:5]

FileNotFoundError: [Errno 2] No such file or directory: './heisSurvey/HEISBalanceSheet_NoCapital.xls'

In [109]:
x = pd.read_stata('./heisSurvey/2015-16/sec_9a.dta')
x["hhcode"] = x["hhcode"].astype(int)
x

,hhcode,province,region,psu,sec,idc,bs1qc1,bs1qc2,bs1qc3,bs1qc4,bs1qc5,bs1qc6,bs1qc7,bs1qc8
0,1102010101,kp,urban,11020101,09A,1,35000.0,12.0,420000.0,NaN,NaN,NaN,NaN,420000.0
1,1102010101,kp,urban,11020101,09A,2,5000.0,12.0,60000.0,NaN,NaN,NaN,NaN,60000.0
2,1102010101,kp,urban,11020101,09A,99,NaN,NaN,480000.0,NaN,NaN,NaN,NaN,480000.0
3,1102010102,kp,urban,11020101,09A,1,15500.0,12.0,186000.0,NaN,NaN,NaN,NaN,186000.0
4,1102010102,kp,urban,11020101,09A,2,NaN,NaN,NaN,NaN,NaN,NaN,108000.0,108000.0
5,1102010102,kp,urban,11020101,09A,99,NaN,NaN,186000.0,NaN,NaN,NaN,108000.0,294000.0
6,1102010103,kp,urban,11020101,09A,1,NaN,NaN,66880.0,NaN,NaN,NaN,113640.0,180520.0
7,1102010103,kp,urban,11020101,09A,2,10000.0,12.0,120000.0,NaN,NaN,NaN,NaN,120000.0
8,1102010103,kp,urban,11020101,09A,99,NaN,NaN,186880.0,NaN,NaN,NaN,113640.0,300520.0
9,1102010104,kp,urban,11020101,09A,51,NaN,NaN,12900.0,NaN,NaN,NaN,NaN,12900.0


In [106]:
heisSection1B_IncomeNoCapital = pd.read_stata("./heisSurvey/2015-16/other_sections/sec_1b.dta", convert_categoricals=False)
heisSection1B_IncomeNoCapital = heisSection1B_IncomeNoCapital.rename(index=str, columns={
                                "s1bq01": "did_work", 
                                "s1bq02": "days_worked", 
                                "s1bq03": "did_have_job", 
                                "s1bq04": "nature_of_occupation",
                                "s1bq05": "nature_of_enterprise",
                                "s1bq06": "employment_status",
                                "s1bq07": "monthly_or_annual",
                                "s1bq08": "monthly_earnings",
                                "s1bq09": "months_worked",
                                "s1bq10": "annual_earnings",
                                "s1bq11": "other_work",
                                "s1bq12": "nature_of_occupation_other",
                                "s1bq13": "nature_of_employer",
                                "s1bq14": "employment_status",
                                "s1bq15": "annual_earnings_other",
                                "s1bq16": "other_other_work",
                                "s1bq17": "annual_earnings_other_other",
                                "s1bq18": "sell_wages",
                                "s1bq19": "money_received_by_selling_wages",
                                "s1bq20": "was_pension_received",
                                "s1bq21": "pension_amount",
                                "s1bq22": "used_for_hh",
                                "s1bq23": "operation_in_agriculture",
                                "s1bq24": "did_own_livestock",
                                "s1bq25": "proprietor_or_partner_of_business",
                                "s1bq26": "how_many_shops_associated"})
# heisSection1B_IncomeNoCapital = heisSection1B_IncomeNoCapital.join(heisSectionPList_Weights.set_index('hhcode'), on='hhcode')
heisSection1B_IncomeNoCapital["hhcode"] = heisSection1B_IncomeNoCapital["hhcode"].astype(int)
heisSection1B_IncomeNoCapital

,hhcode,province,region,psu,sec,idc,did_work,days_worked,did_have_job,nature_of_occupation,nature_of_enterprise,employment_status,monthly_or_annual,monthly_earnings,months_worked,annual_earnings,other_work,nature_of_occupation_other,nature_of_employer,employment_status,annual_earnings_other,other_other_work,annual_earnings_other_other,sell_wages,money_received_by_selling_wages,was_pension_received,pension_amount,used_for_hh,operation_in_agriculture,did_own_livestock,proprietor_or_partner_of_business,how_many_shops_associated
0,1102010101,1,2,11020101,01B,1,1,30.0,NaN,5413.0,8423.0,4.0,1.0,35000.0,12.0,NaN,2.0,NaN,NaN,NaN,NaN,2,NaN,2,NaN,2,NaN,1,2.0,4.0,2.0,NaN
1,1102010101,1,2,11020101,01B,2,1,26.0,NaN,7231.0,4100.0,4.0,1.0,5000.0,12.0,NaN,2.0,NaN,NaN,NaN,NaN,2,NaN,2,NaN,2,NaN,1,NaN,NaN,NaN,NaN
2,1102010101,1,2,11020101,01B,3,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN,2,NaN,3,NaN,NaN,NaN,NaN
3,1102010101,1,2,11020101,01B,51,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN,2,NaN,3,2.0,4.0,2.0,NaN
4,1102010101,1,2,11020101,01B,54,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN,2,NaN,3,NaN,NaN,NaN,NaN
5,1102010101,1,2,11020101,01B,55,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN,2,NaN,3,NaN,NaN,NaN,NaN
6,1102010102,1,2,11020101,01B,1,1,30.0,NaN,5419.0,8510.0,4.0,1.0,15500.0,12.0,NaN,2.0,NaN,NaN,NaN,NaN,2,NaN,2,NaN,2,NaN,1,2.0,4.0,2.0,NaN
7,1102010102,1,2,11020101,01B,2,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN,1,108000.0,1,NaN,NaN,NaN,NaN
8,1102010102,1,2,11020101,01B,51,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN,2,NaN,3,2.0,4.0,2.0,NaN
9,1102010103,1,2,11020101,01B,1,1,20.0,NaN,6330.0,150.0,6.0,2.0,NaN,NaN,66880.0,2.0,NaN,NaN,NaN,NaN,2,NaN,2,NaN,1,113640.0,1,1.0,NaN,2.0,NaN


### Miscellaneous Datasets

In [ ]:
pak_world_bank_indicators = pd.read_excel("./miscData/world_bank_bottom20_income_share_and_population.xlsx")

In [ ]:
lowest_20_income = pak_world_bank_indicators[pak_world_bank_indicators["Series Code"] == "SI.DST.FRST.20"]
lowest_20_income

In [ ]:
population = pak_world_bank_indicators[pak_world_bank_indicators["Series Code"] == "SP.POP.TOTL"]
population

In [ ]:
heisSection1B_IncomeNoCapital.shape 

**Getting a feel for the data**

In [ ]:
# heisSection1B_IncomeNoCapital[pd.notnull(heisSection1B_IncomeNoCapital["money_received_by_selling_wages"])]
heisSection1B_IncomeNoCapital[heisSection1B_IncomeNoCapital["hhcode"] == 1131010201]

In [ ]:
heisSection9A_IncomeNoCapital.shape

In [ ]:
heisSection9A_IncomeNoCapital[heisSection9A_IncomeNoCapital["hhcode"] == 1131010201]

In [ ]:
heisSection9A_IncomeNoCapital

In [ ]:
heisSection9E_IncomeWithCapital.shape

In [ ]:
heisSection9E_IncomeWithCapital[0:10]

**Making sense of the difference in number of entries** (also looking at household survey linked here: http://www.pbs.gov.pk/sites/default/files//pslm/publications/pslm_microdata_2015_16_HIES/HIICS%20Male%20Questionnaire%20%282015-16%29.pdf)

Here is the introduction that came with the survey and explains some of the terms / sampling methods: http://www.pbs.gov.pk/sites/default/files//pslm/publications/pslm_microdata_2015_16_HIES/Introduction.pdf

In [ ]:
# Let's see different entries on the same table
hhcode = 2513311004

# 1 1102020203
# 2 1302032607
# 3 1423211302
# 4 1423212903
# 5 1423215304
# 6 2513311004
# 7 2602010208
# 8 2723241109
# 9 2831010206
# 10 3111030111
# 11 3302011512
# 12 3421040201
# 13 3513211601
# 14 3523220412
# 15 3543221707
# 16 3553221313
# 17 4401020701

In [ ]:
heisSection1B_IncomeNoCapital[heisSection1B_IncomeNoCapital["hhcode"] == hhcode]
# heisSection1B_IncomeNoCapital.psu

In [ ]:
heisSection9A_IncomeNoCapital[heisSection9A_IncomeNoCapital["hhcode"] == hhcode]

In [ ]:
heisSection9E_IncomeWithCapital[heisSection9E_IncomeWithCapital["hhcode"] == hhcode]

In [ ]:
# Test assumption: how many unique households does the first dataset have?
len(set(heisSection1B_IncomeNoCapital["hhcode"]))
# YES!

In [ ]:
households_with_toolow_income = list(heisSection9A_IncomeNoCapital[heisSection9A_IncomeNoCapital["greater_than_0.85"] == "no"]["hhcode"])
households_with_capital_income = list(heisSection9E_IncomeWithCapital["hhcode"])

In [ ]:
print(len(households_with_toolow_income))
print(len(households_with_capital_income))

Households to explore in the last two datasets. It looks like everything that was *heisSection9A_IncomeNoCapital* and didn't have a greater_than_0.85 was re-evaluated in the  *heisSection9E_IncomeWithCapital* section. Let's see if there are any 'no's in the first that weren't evaluated and any households in the second that were evaluated regardless.

In [ ]:
i = 1
for h in households_with_toolow_income:
    if h not in households_with_capital_income:
        print(i, h)
        i += 1
print("\n")
j = 1
for h in households_with_capital_income:
    if h not in households_with_toolow_income:
        print(j, h)
        j += 1

Using the above evaluator for each household to see the values - it looks like 2721040211 has improper data entry for section 9E (therefore I am removing it). The others were simply not followed up on, but the sample size is also so insiginificant that I'm keeping them in here.

**Dataset Descriptions**

In [ ]:
heisSection1B_IncomeNoCapital.shape 
# Income by Individual (for every household, most granular)

In [ ]:
heisSection9A_IncomeNoCapital.shape 
# Income By Household

In [ ]:
heisSection9E_IncomeWithCapital.shape 
# Households above with suspect income / expenditure ratios (adds in capital component for some households).

# When combining the datasets, let's double check to see that these households are mainly in the 
# top 20% of income once adding in their capital earnings. Let's also see where they lie before adding in 
# capital / this dataset's values. TODO need to ask if this is typical and how exactly to interpret this data
# as some respondants seem to be able refuse to do this part (like 2513311004 who seems to be a top 20% earner)

One thing to be cognizant about is that section 1B is filled out by male and female, but section 9 is filled by males only. There are incentives both ways that I need to take into account - it means that there could either be a bias upward to make the family stand out as richer (without the other members of the family in the room to make the person feel accountable) or poorer.

TODO ask professors about potential misaligned incentives due to survey design
TODO pensions are government transfers, "Government  benefits,  reported  in  the  survey,  are excluded from the analysis for consistency with tax tabulations;our focus is pre-tax income."

### Cleaning, Combining and Ordering Datasets

In [ ]:
income_data = [int(x) for x in heisSection9A_IncomeNoCapital["total_income_x"] if not pd.isnull(x)]
weights = [int(x) for x in heisSection9A_IncomeNoCapital["weights"]]
weighted_income_data = []
for i in range(0, len(income_data)):
    income = [income_data[i]]
    w = weights[i]
    weighted_income_data.extend(income * w)

s_data = np.array(sorted(weighted_income_data))
n = len(s_data)
percentiles = 100 * (np.arange(1, n+1) - 0.5) / n

In [ ]:
sum(s_data) / 26581120581632

In [ ]:
[int(x) for x in heisSection9E_IncomeWithCapital["total_income_xy"] if not pd.isnull(x)]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%store -r export_data
%store -r t_percentiles

In [ ]:
# let's compare the data above the 77.5th percentile
other_s_data = s_data[n-5800000:]
n = len(other_s_data)
other_percentiles = 100 * (np.arange(1, n+1) - 0.5) / n

In [ ]:
# Plots of just the data above 400,000 PRK earnings or the top 23% of earners in Pakistan -- EVEN THE TAXPAYER DATA IS BETTER
plt.scatter(other_percentiles[:-10000], other_s_data[:-10000], c='b')
plt.scatter(t_percentiles[:-10000], export_data[:-10000], c='r')
plt.show()

In [ ]:
# Plots of just the data of the top 1% of 20% -- EVEN THE TAXPAYER DATA IS BETTER
plt.scatter(other_percentiles[-10000:], other_s_data[-10000:], c='b')
plt.scatter(t_percentiles[-10000:], export_data[-10000:], c='r')
plt.show()

In [ ]:
plt.scatter(percentiles, s_data)
plt.show()

In [ ]:
s_data[n-5800000]
percentiles[n-5800000]

In [ ]:
plt.scatter(percentiles[n - 5800000:n-10000], s_data[n - 5800000:n-10000])
plt.show()

In [ ]:
# how to combine tax data in individual level... what if we graph it and see the difference...

In [ ]:
s_data = np.array(sorted([int(x) for x in heisSection9E_IncomeWithCapital["total_expenditure_ze"] if not pd.isnull(x)]))
n = len(s_data)
percentiles = 100 * (np.arange(1, n+1) - 0.5) / n

In [ ]:
s_data = np.array(sorted([float(x) for x in heisSection9A_IncomeNoCapital["ratio"] if not pd.isnull(x)]))
n = len(s_data)
percentiles = 100 * (np.arange(1, n+1) - 0.5) / n

In [ ]:
# income / expenditure
plt.scatter(percentiles[n - 7000:n - 2000], s_data[n - 7000:n - 2000])
plt.show()

In [ ]:
# taxes start at 1,200,000, used to start at 400,000 at time of survey
s_data = np.array(sorted([int(x) for x in heisSection9A_IncomeNoCapital["total_income_x"] if not pd.isnull(x)]))
n = len(s_data)
percentiles = 100 * (np.arange(1, n+1) - 0.5) / n
obs_num = 17620
print(s_data[obs_num])
print(percentiles[obs_num])

**Expanding the Survey By Multiplying by Weights**

In [ ]:
# let's combine section 9A and 9E in the surveys

In [ ]:
heisSection9E_IncomeWithCapital

# Exploratory Data Analysis (Visualization)